# Calephelis

## Coperation with Jean-Michel, virus

### find virus related sequences

```
import pandas as pd
import os
import glob

folderBioHPC = '/work/archive/biophysics/Nick_lab/s185491/Celastrina/fq'
folder_fq = '/archive/butterfly/ready_fastq/'

file_summary = '/home/xcao/w/2018Calephelis/summary/20181217Calephelis.xlsx'
df_summary = pd.read_excel(file_summary,sheet_name='Sheet1')
sample_prefix = [e[1:] for e in df_summary['ID']]
print('unique sample prefix', len(set(sample_prefix)))

files_fq = glob.glob(folder_fq+'*')
files_use = [e for e in files_fq if os.path.basename(e).split('_')[0] in sample_prefix]
prefix_fq = [os.path.basename(e).split('_')[0] for e in files_use]

sample_prefix =set(prefix_fq)

txt = '''cd {workingfolder} && mkdir {name} && cd {name} && /home/xcao/p/mira/mira_4.0.2_linux-gnu_x86_64_static/bin/mirabait -k 21 {baitTarget} /archive/butterfly/ready_fastq/{name}_R1.fastq {name}_R1.fastq && /home/xcao/p/mira/mira_4.0.2_linux-gnu_x86_64_static/bin/mirabait -k 21 {baitTarget} /archive/butterfly/ready_fastq/{name}_R2.fastq {name}_R2.fastq && /home/xcao/p/SPAdes/SPAdes-3.13.0-Linux/bin/spades.py -t 4 --s1 {name}_R1.fastq --s2 {name}_R2.fastq -o ./spades/ && mv ./spades/contigs.fasta ../{name}_contig.fa && rm -rf {workingfolder}{name}/spades && rm {workingfolder}{name}/hashstat.bin
'''

workingfolder = '/home/xcao/w/2018Calephelis/20181217virus/scaffold1380/'
baitTarget = '/home/xcao/w/2018Calephelis/20181217virus/scaffold1380/scaffold1308.fa'
open(workingfolder+'jobs.cmds','w').write(''.join(txt.format(workingfolder=workingfolder, baitTarget=baitTarget, name = name) for name in sample_prefix))

baitTarget='/home/xcao/w/2018Calephelis/20181217virus/scaffold6262/scaffold6262.fa'
workingfolder='/home/xcao/w/2018Calephelis/20181217virus/scaffold6262/'
open(workingfolder+'jobs.cmds','w').write(''.join(txt.format(workingfolder=workingfolder, baitTarget=baitTarget, name = name) for name in sample_prefix))

#nohup python /home/xcao/p/xiaolongTools/multiThread.py 48 /home/xcao/w/2018Calephelis/20181217virus/scaffold1380/jobs.cmds &
#nohup python /home/xcao/p/xiaolongTools/multiThread.py 48 /home/xcao/w/2018Calephelis/20181217virus/scaffold6262/jobs.cmds &

folder = '/home/xcao/w/2018Calephelis/20181217virus/scaffold6262/'
contigs_prefix = glob.glob(folder+'*_contig.fa')
contigs_prefix = [os.path.basename(e).split('_')[0] for e in contigs_prefix]
print([e for e in sample_prefix if e not in contigs_prefix])
#the missing ones are ['3505', '3540', '3585', '3847']
missing_prefix = [e for e in sample_prefix if e not in contigs_prefix]
baitTarget='/home/xcao/w/2018Calephelis/20181217virus/scaffold6262/scaffold6262.fa'
workingfolder='/home/xcao/w/2018Calephelis/20181217virus/scaffold6262/'
txt = '''cd {workingfolder} && rm -rf {workingfolder}{name} && mkdir {name} && cd {name} && /home/xcao/p/mira/mira_4.0.2_linux-gnu_x86_64_static/bin/mirabait -k 21 {baitTarget} /archive/butterfly/ready_fastq/{name}_400_R1.fastq {name}_R1.fastq && /home/xcao/p/mira/mira_4.0.2_linux-gnu_x86_64_static/bin/mirabait -k 21 {baitTarget} /archive/butterfly/ready_fastq/{name}_400_R2.fastq {name}_R2.fastq && /home/xcao/p/SPAdes/SPAdes-3.13.0-Linux/bin/spades.py -t 4 --s1 {name}_R1.fastq --s2 {name}_R2.fastq -o ./spades/ && mv ./spades/contigs.fasta ../{name}_contig.fa && rm -rf {workingfolder}{name}/spades && rm {workingfolder}{name}/hashstat.bin
'''
for name in missing_prefix:
    print(txt.format(workingfolder=workingfolder, baitTarget=baitTarget, name = name))

```

## summary

### check existence of files

#### fastq
```
import pandas as pd
import os
import glob

folderBioHPC = '/work/archive/biophysics/Nick_lab/s185491/Celastrina/fq'
folder_fq = '/archive/butterfly/ready_fastq/'
folder_summary = '/home/xcao/w/2018Calephelis/summary/'

file_summary = os.path.join(folder_summary,'20181217Calephelis.xlsx')
df_summary = pd.read_excel(file_summary,sheet_name='Sheet1')
sample_prefix = [e[1:] for e in df_summary['ID']]
print('unique sample prefix', len(set(sample_prefix)))

files_fq = glob.glob(folder_fq+'*')
files_use = [e for e in files_fq if os.path.basename(e).split('_')[0] in sample_prefix]

files_possible = []
for e in files_fq:
    if any([os.path.basename(e).startswith(s) for s in sample_prefix]):
        if e not in files_use:
            files_possible.append(e)
print(files_possible,'possible abnormal name, if not sampleID_R1/R2.fastq')

dc_fq=dict([e,[]] for e in sample_prefix)
for e in files_use:
    dc_fq[os.path.basename(e).split('_')[0]].append(e)

sample_fq = [e for e,v in dc_fq.items() if len(v)==3]
open(os.path.join(folder_summary, 'samples_with_fq.txt'),'w').write('\n'.join(sample_fq))

from collections import Counter
print(Counter([len(e) for e in dc_fq.values()]))
```

284 unique sample names, 266 of them have good fastq files. 18 have no fastq files

#### map files

```
#check existance of map files
files_map = glob.glob('/archive/butterfly/maps/debiased/*.map')
files_map = set([os.path.basename(e) for e in files_map])
print('number of map files',len(files_map))
sample_map = [e for e in files_map if os.path.basename(e).split('_')[0] in sample_prefix]
print(len(sample_map),'map files use')

dc_map = dict([e,[]] for e in sample_prefix)
for e in sample_map:
    dc_map[os.path.basename(e).split('_')[0]].append(e)

print(Counter([len(e) for e in dc_map.values()]))
#get reference genomes
ls_reference = []
for e in sample_map:
    ls_reference.append(os.path.basename(e).split('_',1)[1])
print('reference Counter\n',Counter(ls_reference))

#use 3574_assembly_v1_withMito_snp_step2.map as reference
ref_genome = '3574_assembly_v1_withMito'
sample_map = [e for e in sample_prefix if e+'_'+ref_genome+'_snp_step2.map' in files_map]
files_use_map = ['/archive/butterfly/maps/debiased/' + e+'_'+ref_genome+'_snp_step2.map' for e in sample_map]
print(len(sample_map),'map files use')
open(os.path.join(folder_summary, 'mapfiles.txt'),'w').write('\n'.join(files_use_map))

print(len(set(sample_fq + sample_map)))
#all sample_map is in sample_fq
sample_todo = [e for e in sample_fq if e not in sample_map]
print([e for e in sample_fq if e not in sample_map])

#check existance of statistics files
files = glob.glob('/archive/butterfly/unbias_pipeline_info/step4_postprocessing/final_stats/*.report')
files = set([os.path.basename(e) for e in files])
sample_stats = [e for e in sample_prefix if e+'_'+ref_genome+'_stat.report' in files]
print(len(sample_stats),'file stats')
print('files missing',[e for e in sample_prefix if e not in sample_map])
```

after checking with Qian, 3574 is a better reference genome.
20181224 checked. All 266 file exists

### summarize stat files

```
ls_file = ['/archive/butterfly/unbias_pipeline_info/step4_postprocessing/final_stats/' + e+'_'+ref_genome+'_stat.report' for e in sample_prefix if e+'_'+ref_genome+'_stat.report' in files]
import pandas as pd
df_align_stat = pd.concat([pd.read_csv(e,sep='\t',dtype=str) for e in ls_file],ignore_index=True)
df_align_stat['sample'] =df_align_stat['sample'].apply(lambda x:'S'+x)
print(df_align_stat.shape)
df_align_stat.head()
df_align_stat.to_csv('/home/xcao/w/2018Calephelis/summary/20181225Calephelis_align_stat.csv',sep='\t')
```

### map files to npInt8

```
files_map = '/home/xcao/w/2018Calephelis/summary/mapfiles.txt'
outfolder1 = '/home/xcao/w/2018Calephelis/summary/map.npInt8.0'
outfolder2 = '/home/xcao/w/2018Calephelis/summary/map.npInt8.2'

import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/mapFile')
import mapfileIO
mapfileIO.read2Int8s(filenames=files_map, strand=0, changeGap=True, threads=32, outfolder=outfolder1)
mapfileIO.read2Int8s(filenames=files_map, strand=2, changeGap=True, threads=32, outfolder=outfolder2)
```

### Count gaps in each position

```
import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/mapFile')
import mapfileIO
import glob
import numpy as np
import pickle

folder = '/home/xcao/w/2018Calephelis/summary/map.npInt8.0/'
outfile = '/home/xcao/w/2018Calephelis/summary/map.sitesInfo/gapCount266'
outfileHist = outfile + '.hist'
files = glob.glob(folder+'*')
print('total map files', len(files))

#ls_mapInt8 = mapfileIO.loadMapBinaries(files, threads=8)#multithreading not working. use the slow method
ls_mapInt8 = [mapfileIO.loadMapBinary(f) for f in files]
temp = ls_mapInt8[0]
maplen = len(temp)
gapCount = np.zeros(maplen,dtype=np.int16)
for e in ls_mapInt8:
    gapCount += (e==45)

with open(outfile,'wb') as f:
    pickle.dump(gapCount,f)

gapHist = np.unique(gapCount,return_counts=True)
with open(outfileHist,'w') as f:
    for c, p in zip(*gapHist):
        f.write('{}\t{}\n'.format(c,p))

#average gap per site
gapAvg = gapCount.sum() / maplen
print('average gap per site', gapAvg)
```

### get some information about the genome

#### genome header, scaffold length

```
filename = '/home/xcao/w/genomes/Calephelis/3574_assembly_v1_withMito.fa'
from Bio import SeqIO
fout = open(filename + '.head','w')
for _e in SeqIO.parse(filename,'fasta'):
    fout.write(_e.description+'\n')
fout.close()
fout = open(filename +'.len','w')
for _e in SeqIO.parse(filename,'fasta'):
    fout.write(_e.id+'\t'+str(len(_e.seq))+'\n')
fout.close()
```

#### positions of mitochondria, mRNA, CDS
for the gff file by Qian, there are only mRNA, CDS. So we can group the regions in the genome to 4 groups:  
mitochondria: 1  #Note: 20181226: just found that though the reference contains mitochondrial sequences, the map file does not contain.
intron : 2  
CDS/exon: 6  
inter-gene: 0  
intron+CDS forms gene_region: 2 or 6

```
file_gff = '/home/xcao/w/genomes/Calephelis/3574_assembly_v1.gff'
file_genome_len = '/home/xcao/w/genomes/Calephelis/3574_assembly_v1_withMito.fa.len'

import numpy as np
import pandas as pd
import pickle

df_scf = pd.read_csv(file_genome_len,sep='\t', header=None)
df_scf.columns = ['scf','scf_len']
df_scf = df_scf.iloc[:-1] #remove the mitochondrial row
genomelen = df_scf['scf_len'].sum()

df_scf['start'] = 0
df_scf['end'] = df_scf['scf_len']
for i in df_scf.index[1:]:
    df_scf.loc[i,'start'] = df_scf.loc[i-1,'start'] + df_scf.loc[i-1, 'scf_len']
    df_scf.loc[i,'end'] = df_scf.loc[i,'start'] + df_scf.loc[i,'scf_len']

dc_scf2start = dict(zip(df_scf['scf'], df_scf['start']))
dc_scf2end = dict(zip(df_scf['scf'], df_scf['end']))

df_gff = pd.read_csv(file_gff,sep='\t', header=None)
df_gff.columns = ['scf', 'program', 'type', 'start', 'end', 'special', 'strand','exonN','transcript_id']
df_gff = df_gff.set_index('scf')

gffnp = np.zeros(genomelen,dtype=np.uint8)
##set mitochondrial sites to 1
#start = dc_scf2start['3574_mito']
#end = dc_scf2end['3574_mito']
#gffnp[range(start,end)] = 1
#set mRNA region to 2
for scf, scf_gff in df_gff.iterrows():
    start = scf_gff['start']-1
    end = scf_gff['end']
    start += dc_scf2start[scf]
    end += dc_scf2start[scf]
    if scf_gff['type'] == 'mRNA':
        gffnp[range(start,end)] = 2
#set CDS region to 6
for scf, scf_gff in df_gff.iterrows():
    start = scf_gff['start']-1
    end = scf_gff['end']
    start += dc_scf2start[scf]
    end += dc_scf2start[scf]
    if scf_gff['type'] == 'CDS':
        gffnp[range(start,end)] = 6

with open('/home/xcao/w/2018Calephelis/summary/map.sitesInfo/gff.0inter.2intron.6CDS','wb') as f:
    pickle.dump(gffnp,f)
```

#### get location of exons in CDS alignment
```
file_gff = '/home/xcao/w/genomes/Calephelis/3574_assembly_v1.gff'
file_genome_len = '/home/xcao/w/genomes/Calephelis/3574_assembly_v1_withMito.fa.len'

import numpy as np
import pandas as pd
import pickle

df_scf = pd.read_csv(file_genome_len,sep='\t', header=None)
df_scf.columns = ['scf','scf_len']
df_scf = df_scf.iloc[:-1] #remove the mitochondrial row
genomelen = df_scf['scf_len'].sum()

df_scf['start'] = 0
df_scf['end'] = df_scf['scf_len']
for i in df_scf.index[1:]:
    df_scf.loc[i,'start'] = df_scf.loc[i-1,'start'] + df_scf.loc[i-1, 'scf_len']
    df_scf.loc[i,'end'] = df_scf.loc[i,'start'] + df_scf.loc[i,'scf_len']

dc_scf2start = dict(zip(df_scf['scf'], df_scf['start']))
dc_scf2end = dict(zip(df_scf['scf'], df_scf['end']))

df_gff = pd.read_csv(file_gff,sep='\t', header=None)
df_gff.columns = ['scf', 'program', 'type', 'start', 'end', 'special', 'strand','phase','transcript_id']
df_gff['map_start'] = df_gff.apply(lambda x:dc_scf2start[x['scf']] + x['start']-1, axis = 'columns')
df_gff['map_end'] = df_gff.apply(lambda x:dc_scf2start[x['scf']] + x['end'], axis = 'columns')

with open('/home/xcao/w/2018Calephelis/summary/map.sitesInfo/gff.0inter.2intron.6CDS','rb') as f:
    gffnp = pickle.load(f)

df_exon = df_gff[df_gff['type'] == 'CDS'].copy()


npExonDC = -np.ones(genomelen, dtype=np.int64)#the same length as the genome.
start = 0
for n,v in enumerate(gffnp):
    if v == 6:
        npExonDC[n] = start
        start += 1

df_exon['cds_start'] = df_exon['map_start'].apply(lambda x: npExonDC[x])
df_exon['cds_end'] = df_exon['map_end'].apply(lambda x: npExonDC[x-1]+1)
df_exon['exon_N'] = df_exon.groupby(['transcript_id']).cumcount()+1

fout = open('/home/xcao/w/genomes/Calephelis/3574_assembly_v1_withMito.alignment.exon.loc','w')
for _n, row in df_exon.iterrows():
    fout.write('{transcript_id}.e{exon_n}\t{cds_start}\t{cds_end}\n'.format(transcript_id = row['transcript_id'], exon_n = row['exon_N'], cds_start = row['cds_start'], cds_end = row['cds_end']))
fout.close()




```

### split the genome to fragments

split the genome according to different scan window. get a dictionary with name as key and np array of position in genome as value


12140 fragments for window_size 50k  
4893 fragments for window_size 100k  

```
file_genome_info = '/home/xcao/w/genomes/Calephelis/3574_assembly_v1_withMito.fa.len'
window_size = 50000 #split the genome to parts >= window_size but < 2*window_size. 
fout = '/home/xcao/w/2018Calephelis/summary/map.sitesInfo/dc_sites_window50000'

import pandas as pd
import numpy as np
import pickle

df_genome = pd.read_csv(file_genome_info,sep='\t',header=None)
df_genome.columns = ['scf','scf_len']
df_genome = df_genome[:-1] #drop the row with mitochondria
print('genome size is', df_genome['scf_len'].sum()) #genome size is 808598630

#get start and end of each scaffold in map file
starts = [0]
for scf_len in df_genome['scf_len']:
    starts.append(starts[-1] + scf_len)
df_genome['start'] = starts[:-1]
df_genome['end'] = starts[1:]

#filter df_genome, keep those only larger than window_size
df_filter = df_genome[df_genome['scf_len']>=window_size] #3733 outof 40395 scaffolds left

#determine th splittig parts for each scaffold
df_filter['parts'] = df_filter['scf_len'] // window_size

#generate a dictionary with sequence name and locations in map file
dc_frags = {}
for n, row in df_filter.iterrows():
    scf_positions = np.array(range(row['start'], row['end']), dtype=np.int32)
    ls_scf_positions = np.array_split(scf_positions,row['parts'])
    for frag_n, frag_position in enumerate(ls_scf_positions):
        frag_id = row['scf'] + '_' +str(frag_n)
        dc_frags[frag_id] = frag_position

with open(fout,'wb') as f:
    pickle.dump(dc_frags, f)

```

## Phylogenetic Trees

### identify z-chromsomes

#### run promer3
```
workfolder = '/home/xcao/w/2018Calephelis/20181225zchromosome/'
target = 'Calephelis3574'
target_genome = '/home/xcao/w/genomes/Calephelis/3574_assembly_v1_withMito.fa'
ls_reference = ['Spodopetera', 'Heliconius','Bombyx']
ls_reference_genome=['/home/xcao/w/genomes/Spodoptera_litura/GCF_002706865.1_ASM270686v1_genomic.fna', '/home/xcao/w/genomes/Heliconius_melpomene/Heliconius_melpomene_melpomene_Hmel2.5.scaffolds.fa', '/home/xcao/w/genomes/Bombyx_mori/Bomo_genome_assembly.fa']

commands = '''
cd {workfolder} &&\
/home/xcao/p/mummer/MUMmer3.23/promer  -p {target}_2_{reference}.promer3 {reference_genome} {target_genome} &&\
/home/xcao/p/mummer/MUMmer3.23/delta-filter -q -r {target}_2_{reference}.promer3.delta >{target}_2_{reference}.promer3.delta.qr &&\
/home/xcao/p/mummer/MUMmer3.23/mummerplot {target}_2_{reference}.promer3.delta.qr -t postscript -p {target}_2_{reference}.promer3.delta.qr &&\
/home/xcao/p/mummer/MUMmer3.23/show-coords -rcl {target}_2_{reference}.promer3.delta.qr >{target}_2_{reference}.promer3.coords.qr &
'''

for reference, reference_genome in zip(ls_reference, ls_reference_genome):
    print(commands.format(reference=reference,reference_genome=reference_genome,target=target,target_genome=target_genome,workfolder=workfolder))
```

####  assign scaffold of Celastrina according to 'Spodopetera', 'Heliconius','Bombyx'

```
import pandas as pd
import numpy as np
import os

files = '''/home/xcao/w/2018Calephelis/20181225zchromosome/Calephelis3574_2_Bombyx.promer3.coords.qr
/home/xcao/w/2018Calephelis/20181225zchromosome/Calephelis3574_2_Heliconius.promer3.coords.qr
/home/xcao/w/2018Calephelis/20181225zchromosome/Calephelis3574_2_Spodopetera.promer3.coords.qr
'''.split()
filename_coord = files[2]

filename = filename_coord
l = open(filename).readlines()
ls =[e.split() for e in l[5:]]

df_coord = pd.DataFrame(ls)
df_coord = df_coord[[0,1,3,4,6,7,9,10,11,13,14,16,17,19,20,21,22]].copy()
df_coord[[0,1,3,4,6,7,9,10,11,13,14,16,17,19,20]] = df_coord[[0,1,3,4,6,7,9,10,11,13,14,16,17,19,20]].apply(pd.to_numeric)
df_coord.columns = ['ref_start','ref_end','query_start','query_end','ref_match_len','query_match_len','identity','similarity','percent_stop_codon','ref_len','query_len','ref_coverage','query_coverage','ref_reading_frame','query_reading_frame','ref_id','query_id']
df_coord_group_qr = df_coord.groupby(['query_id','ref_id'])

df_qr = df_coord_group_qr['ref_match_len'].sum()
df_summary = pd.DataFrame([ [e,df_qr[e].idxmax(),df_qr[e].max(),df_qr[e].sum(), df_qr[e].max()/df_qr[e].sum()] for e in df_qr.index.unique(0)])
df_summary.columns = ['test_id','ref_id','scaffold_match_len','total_match_len','ratio_specific']


for n in range(df_summary.shape[0]):
    temp_df = df_coord_group_qr.get_group((df_summary['test_id'][n],df_summary['ref_id'][n]))
    match_fragments = temp_df.shape[0]
    df_summary.loc[n,'match_fragments'] = match_fragments
    positions = []
    for m in range(match_fragments):
        start = temp_df.iloc[m]['ref_start']
        end = temp_df.iloc[m]['ref_end']
        if start > end:
            start, end = end, start
        positions += list(range(start, end))
    position_avg = np.median(positions)
    df_summary.loc[n,'position_avg'] = position_avg


df_summary['match_fragments'] = df_summary['match_fragments'].astype(np.int64)
df_summary['position_avg'] = df_summary['position_avg'].astype(np.int64)
df_summary = df_summary.sort_values(by=['ref_id','position_avg'])

outfile = os.path.join(os.path.dirname(filename_coord), '20181226'+os.path.basename(filename_coord).split('.')[0]+'.csv')
df_summary.to_csv(outfile,sep='\t',index=False)
```

#### get z-chromosome scaffolds according to 'Spodopetera', 'Heliconius','Bombyx'

z-chromosome infor based on (scaf num, total length)  
Bombyx: 188 36080881  
Heliconius: 139 33192451  
Spodopetera: 163 34135513  
all: 87 29582058  
  


```
import pandas as pd

filename = '/home/xcao/w/2018Calephelis/20181225zchromosome/20181226Calephelis3574_2_Bombyx.csv'
df_bom = pd.read_csv(filename,sep='\t')

filename = '/home/xcao/w/2018Calephelis/20181225zchromosome/20181226Calephelis3574_2_Heliconius.csv'
df_hel = pd.read_csv(filename,sep='\t')

filename = '/home/xcao/w/2018Calephelis/20181225zchromosome/20181226Calephelis3574_2_Spodopetera.csv'
df_spo = pd.read_csv(filename,sep='\t')

df_bom_z = df_bom[df_bom['ref_id'] == 'Bomo_Chr1']
df_hel_z = df_hel[df_hel['ref_id'] == 'Hmel221001o']
df_spo_z = df_spo[df_spo['ref_id'] == 'NC_036217.1']

zscf_bom = set(df_bom_z['test_id'])
zscf_hel = set(df_hel_z['test_id'])
zscf_spo = set(df_spo_z['test_id'])
zscf_all = zscf_bom & zscf_hel & zscf_spo

file_genome = '/home/xcao/w/genomes/Calephelis/3574_assembly_v1_withMito.fa'
from Bio import SeqIO
fout = open(file_genome+'.head','w')
for _e in SeqIO.parse(file_genome,'fasta'):
    fout.write(_e.description+'\n')
fout.close()
fout = open(file_genome+'.len','w')
for _e in SeqIO.parse(file_genome,'fasta'):
    fout.write(_e.id+'\t'+str(len(_e.seq))+'\n')
fout.close()

dc_scaflen = {}
filename = file_genome+'.len'
for line in open(filename):
    scf,l = line.split()
    l=int(l)
    dc_scaflen[scf] = l

print('Bombyx:', len(zscf_bom), sum([dc_scaflen[e] for e in zscf_bom]))
print('Heliconius:', len(zscf_hel), sum([dc_scaflen[e] for e in zscf_hel]))
print('Spodopetera:', len(zscf_spo), sum([dc_scaflen[e] for e in zscf_spo]))
print('all:', len(zscf_all), sum([dc_scaflen[e] for e in zscf_all]))

open('/home/xcao/w/2018Calephelis/summary/20181226Calephelis3574_z_scf_basedon3','w').write('\n'.join(zscf_all))
```

#### generate a filter np.array file to extract z-chromosome sequences

npSites is a numpy array with True and False. If True, means a z-chromosome sites based on the three reference species
```
file_genomelen = '/home/xcao/w/genomes/Calephelis/3574_assembly_v1_withMito.fa.len'
file_targetscf = '/home/xcao/w/2018Calephelis/summary/20181226Calephelis3574_z_scf_basedon3'

import numpy as np
import pickle
import itertools
sites = [] #True for keep, False not keep. The same length as the genome
target_scfs = open(file_targetscf).read().split()
target_scfs = set(target_scfs)
ls_genomelen = open(file_genomelen).readlines()[:-1] #remove mitochondrial line
for line in ls_genomelen:
    scf,scflen = line.split()
    scflen = int(scflen)
    if scf in target_scfs:
        sites.append([True] * scflen)
    else:
        sites.append([False] * scflen)
sites = list(itertools.chain.from_iterable(sites))
npSites = np.array(sites)
with open('/home/xcao/w/2018Calephelis/summary/map.sitesInfo/zChrFilter','wb') as f:
    pickle.dump(npSites,f)
```

### make z-chromosme phylogenetic tree

####  extract sequence

```
import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/mapFile')
import mapfileIO
import glob

folder = '/home/xcao/w/2018Calephelis/summary/map.npInt8.0/'
outfile = '/home/xcao/w/2018Calephelis/20181226Phylo/zChrBased3Ref/Calephelis266.zChr'
file_filter = '/home/xcao/w/2018Calephelis/summary/map.sitesInfo/zChrFilter'
files = glob.glob(folder+'*')
print('total map npInt8 files', len(files))

mapfileIO.npInt8stoFastas(files, threads=32, output=outfile, headerFun=lambda x:x.split('_')[0], npFilter=file_filter, npFilterFun=None, merged=True)
```

#### filter sequence

```
#nonGap ratio 0.8
python /home/xcao/p/xiaolongTools/utils/fastaAlignmentFilter.py -i /home/xcao/w/2018Calephelis/20181226Phylo/zChrBased3Ref/Calephelis266.zChr -o /home/xcao/w/2018Calephelis/20181226Phylo/zChrBased3Ref/Calephelis266.zChr.filter0.8 -t 32 -b 0 -g 0.8 -B 0
#nonGap ratio 0.7
python /home/xcao/p/xiaolongTools/utils/fastaAlignmentFilter.py -i /home/xcao/w/2018Calephelis/20181226Phylo/zChrBased3Ref/Calephelis266.zChr -o /home/xcao/w/2018Calephelis/20181226Phylo/zChrBased3Ref/Calephelis266.zChr.filter0.7 -t 32 -b 0 -g 0.7 -B 0
#nonGap ratio 0.6
python /home/xcao/p/xiaolongTools/utils/fastaAlignmentFilter.py -i /home/xcao/w/2018Calephelis/20181226Phylo/zChrBased3Ref/Calephelis266.zChr -o /home/xcao/w/2018Calephelis/20181226Phylo/zChrBased3Ref/Calephelis266.zChr.filter0.6 -t 32 -b 0 -g 0.6 -B 0
#nonGap ratio 0.5
python /home/xcao/p/xiaolongTools/utils/fastaAlignmentFilter.py -i /home/xcao/w/2018Calephelis/20181226Phylo/zChrBased3Ref/Calephelis266.zChr -o /home/xcao/w/2018Calephelis/20181226Phylo/zChrBased3Ref/Calephelis266.zChr.filter0.5 -t 32 -b 0 -g 0.5 -B 0
#nonGap ratio 0.4
python /home/xcao/p/xiaolongTools/utils/fastaAlignmentFilter.py -i /home/xcao/w/2018Calephelis/20181226Phylo/zChrBased3Ref/Calephelis266.zChr -o /home/xcao/w/2018Calephelis/20181226Phylo/zChrBased3Ref/Calephelis266.zChr.filter0.4 -t 32 -b 0 -g 0.4 -B 0
#nonGap ratio 0.3
python /home/xcao/p/xiaolongTools/utils/fastaAlignmentFilter.py -i /home/xcao/w/2018Calephelis/20181226Phylo/zChrBased3Ref/Calephelis266.zChr -o /home/xcao/w/2018Calephelis/20181226Phylo/zChrBased3Ref/Calephelis266.zChr.filter0.3 -t 32 -b 0 -g 0.3 -B 0
```

#### run a quick one with filter 0.8 to get the guider tree for examl
```
txt_qsub = '''#!/bin/bash
#SBATCH --job-name=zChr
#SBATCH --partition=32GB
#SBATCH --nodes=32
#SBATCH --ntasks=1024
#SBATCH --time=5-00:00:00
#SBATCH --output=0.%j.out
#SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL

module load openmpi/intel/3.1.1

/home2/s185491/p/anaconda3/anaconda520/bin/python3 /home2/s185491/p/xiaolongTools/utils/runExaML_withFastaForLargeFiles.py -i {filename} -T 1024
'''

files = ['/work/biophysics/s185491/20181226Calephelis/Phylo/zChr/Calephelis266.zChr.filter0.8']

import os
for filename in files:
    fout = open('/work/biophysics/s185491/20181226Calephelis/Phylo/zChr/cmds/qsub.Calephelis266.zChr0.8','w')
    fout.write(txt_qsub.format(filename=filename,n=os.path.basename(filename)))
    fout.close()
```

### make mitochondrial tree

#### extract sequences

```
import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/mapFile')
import mapfileIO
filename = '/home/xcao/w/2018Calephelis/summary/mapfiles.txt'
ls_files = open(filename).read().split()
files = [f.replace('3574_assembly_v1_withMito_snp_step2.map', '3574_assembly_v1_withMito_snp_step2_MITO.map') for f in ls_files]
outfile = '/home/xcao/w/2018Calephelis/20181226Phylo/mitochondria/Calephelis266.mito'
mapfileIO.mapFilestoFastas(files, threads=32, output=outfile, headerFun=lambda x:x.split('/')[-1].split('_')[0], npFilter=None, npFilterFun=None, merged=True)
```

#### run MrBayes

cp sequence to bioHPC. For runMrBayes_largeInput, change chain to 2, and number of generation to 2 million
```
#!/bin/bash
#SBATCH --job-name=mito
#SBATCH --partition=256GB
#SBATCH --nodes=1
#SBATCH --ntasks=48
#SBATCH --time=5-00:00:00
#SBATCH --output=0.%j.out
#SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL

module load openmpi/intel/3.1.1
module load beagle-lib/2.1.2

python /home2/s185491/p/xiaolongTools/project/Junonia/runMrBayes_largeInput.py -i /work/biophysics/s185491/20181226Calephelis/Phylo/mito/Calephelis266.mito
```

```
cp /work/biophysics/s185491/20181226Calephelis/Phylo/mitoNewick/Calephelis266.mito /work/biophysics/s185491/20181226Calephelis/Phylo/mito/Calephelis266.mito.newick
 cp /work/biophysics/s185491/20181226Calephelis/Phylo/mitoMrBayes/Calephelis266.mito /work/biophysics/s185491/20181226Calephelis/Phylo/mito/Calephelis266.mito.nex
#rename
/home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py /work/biophysics/s185491/20181226Calephelis/Phylo/mito/Calephelis266.mito.newick
```



### make whole genome, CDS, intron tree

#### extract sequences
```
import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/mapFile')
import mapfileIO
import glob

folder = '/home/xcao/w/2018Calephelis/summary/map.npInt8.0/'
outfile = '/home/xcao/w/2018Calephelis/20181226Phylo/whole/Calephelis266.wholeGap60'
file_filter = '/home/xcao/w/2018Calephelis/summary/map.sitesInfo/gapCount266'
files = glob.glob(folder+'*')
print('total map npInt8 files', len(files))

mapfileIO.npInt8stoFastas(files, threads=32, output=outfile, headerFun=lambda x:x.split('_')[0], npFilter=file_filter, npFilterFun=lambda x:x<=60, merged=True)

outfile = '/home/xcao/w/2018Calephelis/20181226Phylo/whole/Calephelis266.wholeGap13'
mapfileIO.npInt8stoFastas(files, threads=32, output=outfile, headerFun=lambda x:x.split('_')[0], npFilter=file_filter, npFilterFun=lambda x:x<=13, merged=True)

file_filter = '/home/xcao/w/2018Calephelis/summary/map.sitesInfo/gff.0inter.2intron.6CDS'
outfile = '/home/xcao/w/2018Calephelis/20181226Phylo/CDS/Calephelis266.CDS'
mapfileIO.npInt8stoFastas(files, threads=32, output=outfile, headerFun=lambda x:x.split('_')[0], npFilter=file_filter, npFilterFun=lambda x:x==6, merged=True)

file_filter = '/home/xcao/w/2018Calephelis/summary/map.sitesInfo/gff.0inter.2intron.6CDS'
outfile = '/home/xcao/w/2018Calephelis/20181226Phylo/intron/Calephelis266.intron'
mapfileIO.npInt8stoFastas(files, threads=32, output=outfile, headerFun=lambda x:x.split('_')[0], npFilter=file_filter, npFilterFun=lambda x:x==2, merged=True)

file_filter1 = '/home/xcao/w/2018Calephelis/summary/map.sitesInfo/gff.0inter.2intron.6CDS'
file_filter2 = '/home/xcao/w/2018Calephelis/summary/map.sitesInfo/gapCount266'
npFilter1 = mapfileIO.loadMapBinary(file_filter1)
npFilter2 = mapfileIO.loadMapBinary(file_filter2)

outfile = '/home/xcao/w/2018Calephelis/20181226Phylo/CDS/Calephelis266.CDS.Gap26'
mapfileIO.npInt8stoFastas(files, threads=32, output=outfile, headerFun=lambda x:x.split('_')[0], npFilter=((npFilter1==6) & (npFilter2<=26)), npFilterFun=None, merged=True)

outfile = '/home/xcao/w/2018Calephelis/20181226Phylo/intron/Calephelis266.intron.Gap26'
mapfileIO.npInt8stoFastas(files, threads=32, output=outfile, headerFun=lambda x:x.split('_')[0], npFilter=((npFilter1==2) & (npFilter2<=26)), npFilterFun=None, merged=True)

outfile = '/home/xcao/w/2018Calephelis/20181226Phylo/intron/Calephelis266.intron.Gap100'
mapfileIO.npInt8stoFastas(files, threads=32, output=outfile, headerFun=lambda x:x.split('_')[0], npFilter=((npFilter1==2) & (npFilter2<=100)), npFilterFun=None, merged=True)
```

#### transfer files
```
scp -r /home/xcao/w/2018Calephelis/20181226Phylo/whole/ s185491@nucleus.biohpc.swmed.edu:/work/biophysics/s185491/20181226Calephelis/Phylo/
```

#### run examl

run examl for the small file to generate the starting tree. Then split the CDS, whole genome and intron sequences to 5 parts, and run ExaML
```
python /home2/s185491/p/xiaolongTools/utils/splitFastaAlignments2NPartsContinuingEqualStepOrRandom.py -t 48 -m 1 -N 5 -i /work/biophysics/s185491/20181226Calephelis/Phylo/CDS/Calephelis266.CDS -o /work/biophysics/s185491/20181226Calephelis/Phylo/CDS/

python /home2/s185491/p/xiaolongTools/utils/splitFastaAlignments2NPartsContinuingEqualStepOrRandom.py -t 48 -m 1 -N 5 -i /work/biophysics/s185491/20181226Calephelis/Phylo/intron/Calephelis266.intron.Gap100 -o /work/biophysics/s185491/20181226Calephelis/Phylo/intron/

python /home2/s185491/p/xiaolongTools/utils/splitFastaAlignments2NPartsContinuingEqualStepOrRandom.py -t 48 -m 1 -N 5 -i /work/biophysics/s185491/20181226Calephelis/Phylo/whole/Calephelis266.wholeGap60 -o /work/biophysics/s185491/20181226Calephelis/Phylo/whole/

```

```
txt_qsub = '''#!/bin/bash
#SBATCH --job-name={key}_examl
#SBATCH --partition=32GB
#SBATCH --nodes=10
#SBATCH --ntasks=320
#SBATCH --time=5-00:00:00
#SBATCH --output=0.%j.out
#SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL

module load openmpi/intel/3.1.1


'''
commandline = '''/home2/s185491/p/anaconda3/anaconda520/bin/python3 /home2/s185491/p/xiaolongTools/utils/runExaML_withFastaForLargeFiles.py -i {filename} -T 320 -r {file_ref}
'''

files = '''/work/biophysics/s185491/20181226Calephelis/Phylo/zChr/Calephelis266.zChr.split5.0
/work/biophysics/s185491/20181226Calephelis/Phylo/zChr/Calephelis266.zChr.split5.1
/work/biophysics/s185491/20181226Calephelis/Phylo/zChr/Calephelis266.zChr.split5.2
/work/biophysics/s185491/20181226Calephelis/Phylo/zChr/Calephelis266.zChr.split5.3
/work/biophysics/s185491/20181226Calephelis/Phylo/zChr/Calephelis266.zChr.split5.4
'''.split()

dc_files = {}
dc_ref = {}
dc_files['CDS'] = '/work/biophysics/s185491/20181226Calephelis/Phylo/CDS/Calephelis266.CDS.split5.{N}'
dc_ref['CDS'] = '/work/biophysics/s185491/20181226Calephelis/Phylo/CDS/ExaML_result.Calephelis266.CDS.Gap26'
dc_files['intron'] = '/work/biophysics/s185491/20181226Calephelis/Phylo/intron/Calephelis266.intron.Gap100.split5.{N}'
dc_ref['intron'] = '/work/biophysics/s185491/20181226Calephelis/Phylo/intron/ExaML_result.Calephelis266.intron.Gap26'
dc_files['whole'] = '/work/biophysics/s185491/20181226Calephelis/Phylo/whole/Calephelis266.wholeGap60.split5.{N}'
dc_ref['whole'] = '/work/biophysics/s185491/20181226Calephelis/Phylo/whole/ExaML_result.Calephelis266.wholeGap13'

for k in dc_files:
    ls_files = [dc_files[k].format(N = i) for i in range(5)]
    file_ref = dc_ref[k]
    fout = open('/work/biophysics/s185491/20181226Calephelis/Phylo/cmds/qsub_examl_'+k,'w')
    fout.write(txt_qsub.format(key=k))
    for filename in ls_files:
        fout.write(commandline.format(filename=filename, file_ref=file_ref))
    fout.close()




```

#### combine whole genome trees
```
 cat /work/biophysics/s185491/20181226Calephelis/Phylo/whole/ExaML_result.Calephelis266.wholeGap60.split5.* >/work/biophysics/s185491/20181226Calephelis/Phylo/whole/ExaML_result.Calephelis266.wholeGap60.all5
 /home2/s185491/p/anaconda3/anaconda520/bin/sumtrees.py -f 0.00 /work/biophysics/s185491/20181226Calephelis/Phylo/whole/ExaML_result.Calephelis266.wholeGap60.all5 >/work/biophysics/s185491/20181226Calephelis/Phylo/whole/ExaML_result.Calephelis266.wholeGap60.all5.sum
 python /home2/s185491/p/xiaolongTools/utils/trees/sumtreesNEXUS2newick.py -i /work/biophysics/s185491/20181226Calephelis/Phylo/whole/ExaML_result.Calephelis266.wholeGap60.all5.sum
 /home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py /work/biophysics/s185491/20181226Calephelis/Phylo/whole/ExaML_result.Calephelis266.wholeGap60.all5.sum.nwk
```

#### combine zChr
```
 cat /work/biophysics/s185491/20181226Calephelis/Phylo/whole/ExaML_result.Calephelis266.wholeGap60.split5.* >/work/biophysics/s185491/20181226Calephelis/Phylo/whole/ExaML_result.Calephelis266.wholeGap60.all5
 /home2/s185491/p/anaconda3/anaconda520/bin/sumtrees.py -f 0.00 /work/biophysics/s185491/20181226Calephelis/Phylo/whole/ExaML_result.Calephelis266.wholeGap60.all5 >/work/biophysics/s185491/20181226Calephelis/Phylo/whole/ExaML_result.Calephelis266.wholeGap60.all5.sum
 python /home2/s185491/p/xiaolongTools/utils/trees/sumtreesNEXUS2newick.py -i /work/biophysics/s185491/20181226Calephelis/Phylo/whole/ExaML_result.Calephelis266.wholeGap60.all5.sum
 /home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py /work/biophysics/s185491/20181226Calephelis/Phylo/whole/ExaML_result.Calephelis266.wholeGap60.all5.sum.nwk
```

#### combine intron
```
 cat /work/biophysics/s185491/20181226Calephelis/Phylo/intron/ExaML_result.Calephelis266.intron.Gap100.split5.* >/work/biophysics/s185491/20181226Calephelis//Phylo/intron/ExaML_result.Calephelis266.intron.Gap100.all5
 /home2/s185491/p/anaconda3/anaconda520/bin/sumtrees.py -f 0.00 /work/biophysics/s185491/20181226Calephelis//Phylo/intron/ExaML_result.Calephelis266.intron.Gap100.all5 >/work/biophysics/s185491/20181226Calephelis//Phylo/intron/ExaML_result.Calephelis266.intron.Gap100.all5.sum
 python /home2/s185491/p/xiaolongTools/utils/trees/sumtreesNEXUS2newick.py -i /work/biophysics/s185491/20181226Calephelis//Phylo/intron/ExaML_result.Calephelis266.intron.Gap100.all5.sum
 /home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py /work/biophysics/s185491/20181226Calephelis//Phylo/intron/ExaML_result.Calephelis266.intron.Gap100.all5.sum.nwk
```

#### combine CDS
```
 cat /work/biophysics/s185491/20181226Calephelis/Phylo/CDS/ExaML_result.Calephelis266.CDS.split5.* >/work/biophysics/s185491/20181226Calephelis/Phylo/CDS/ExaML_result.Calephelis266.CDS.all5
 /home2/s185491/p/anaconda3/anaconda520/bin/sumtrees.py -f 0.00 /work/biophysics/s185491/20181226Calephelis/Phylo/CDS/ExaML_result.Calephelis266.CDS.all5 >/work/biophysics/s185491/20181226Calephelis/Phylo/CDS/ExaML_result.Calephelis266.CDS.all5.sum
 python /home2/s185491/p/xiaolongTools/utils/trees/sumtreesNEXUS2newick.py -i /work/biophysics/s185491/20181226Calephelis/Phylo/CDS/ExaML_result.Calephelis266.CDS.all5.sum
 /home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py /work/biophysics/s185491/20181226Calephelis/Phylo/CDS/ExaML_result.Calephelis266.CDS.all5.sum.nwk
```

### Phylogenetic Tree for C. rawsoni group

#### count gaps

gap ratio 45.69%

```
import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/mapFile')
import mapfileIO
import glob
import numpy as np
import pickle
import pandas as pd
import os

folderBioHPC = '/work/archive/biophysics/Nick_lab/s185491/Celastrina/fq'
folder_fq = '/archive/butterfly/ready_fastq/'
folder_summary = '/home/xcao/w/2018Calephelis/summary/'

file_summary = os.path.join(folder_summary,'20181217Calephelis.xlsx')
df_summary = pd.read_excel(file_summary,sheet_name='Sheet1')
df_summary = df_summary[df_summary["Filter_rawsoni"]==1]
sample_prefix = [e[1:] for e in df_summary['ID']]
print('unique sample prefix', len(set(sample_prefix)))

folder = '/home/xcao/w/2018Calephelis/summary/map.npInt8.0/'
outfile = '/home/xcao/w/2018Calephelis/summary/map.sitesInfo/gapCount.rawsoni35'
outfileHist = outfile + '.hist'
files = glob.glob(folder+'*')
files = [f for f in files if os.path.basename(f).split('_')[0] in sample_prefix]

print('total map files', len(files))

#ls_mapInt8 = mapfileIO.loadMapBinaries(files, threads=8)#multithreading not working. use the slow method
ls_mapInt8 = [mapfileIO.loadMapBinary(f) for f in files]
temp = ls_mapInt8[0]
maplen = len(temp)
gapCount = np.zeros(maplen,dtype=np.int16)
for e in ls_mapInt8:
    gapCount += (e==45)

with open(outfile,'wb') as f:
    pickle.dump(gapCount,f)

gapHist = np.unique(gapCount,return_counts=True)
with open(outfileHist,'w') as f:
    for c, p in zip(*gapHist):
        f.write('{}\t{}\n'.format(c,p))

#average gap per site
gapAvg = gapCount.sum() / maplen
print('average gap per site', gapAvg)
```

#### extract sequences of mito, Z, CDS, whole and intron

previous step continued
```
npFilter_gap = gapCount
npFilter_sites = mapfileIO.loadMapBinary(filename='/home/xcao/w/2018Calephelis/summary/map.sitesInfo/gff.0inter.2intron.6CDS')
dc_files = {os.path.basename(k).split('_')[0]:v for k,v in zip(files, ls_mapInt8)}

outfolder = '/home/xcao/w/2018Calephelis/20190101rawsoni/'
outfile = os.path.join(outfolder,'Calephelis35.wholeGap7')
mapfileIO.npInt8stoFastas(dc_files, threads=32, output=outfile, headerFun=lambda x:x.split('_')[0], npFilter=npFilter_gap, npFilterFun=lambda x:x<=7, merged=True)

outfile = os.path.join(outfolder,'Calephelis35.CDS')
mapfileIO.npInt8stoFastas(dc_files, threads=32, output=outfile, headerFun=lambda x:x.split('_')[0], npFilter=npFilter_sites, npFilterFun=lambda x:x==6, merged=True)


def extractSeq(file_in, file_out, keepIDs):
    '''
    extract sequence from file_in and write to file_out if seq.id in keepIDs
    '''
    count_filein = 0
    count_fileout = 0
    keepIDs = set(keepIDs)
    fout = open(file_out,'w')
    for s in SeqIO.parse(file_in,'fasta'):
        count_filein += 1
        if s.id in keepIDs:
            count_fileout += 1
            fout.write('>'+s.id+'\n'+str(s.seq)+'\n')
    fout.close()
    print('input seqnum:', count_filein,'output seqnum:', count_fileout)
    return None

outfile = os.path.join(outfolder,'Calephelis35.zChr')
file_in = '/home/xcao/w/2018Calephelis/20181226Phylo/zChrBased3Ref/Calephelis266.zChr'
extractSeq(file_in,outfile, sample_prefix)

outfile = os.path.join(outfolder,'Calephelis35.mito')
file_in = '/home/xcao/w/2018Calephelis/20181226Phylo/mitochondria/Calephelis266.mito'
extractSeq(file_in,outfile, sample_prefix)

outfile = os.path.join(outfolder,'Calephelis35.intron')
file_in = '/home/xcao/w/2018Calephelis/20181226Phylo/intron/Calephelis266.intron'
extractSeq(file_in,outfile, sample_prefix)

```

#### transfer files and run tree

transfer
```
scp -r /home/xcao/w/2018Calephelis/20190101rawsoni/ s185491@nucleus.biohpc.swmed.edu:/work/biophysics/s185491/20181226Calephelis/Phylo
```

run mitochondria


mito is simple. Change some words.
```
python /home2/s185491/p/xiaolongTools/utils/splitFastaAlignments2NPartsContinuingEqualStepOrRandom.py -t 48 -m 1 -N 5 -i /work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.CDS -o /work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/

python /home2/s185491/p/xiaolongTools/utils/splitFastaAlignments2NPartsContinuingEqualStepOrRandom.py -t 48 -m 1 -N 5 -i /work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.intron -o /work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/

python /home2/s185491/p/xiaolongTools/utils/splitFastaAlignments2NPartsContinuingEqualStepOrRandom.py -t 48 -m 1 -N 5 -i /work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.wholeGap7 -o /work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/

python /home2/s185491/p/xiaolongTools/utils/splitFastaAlignments2NPartsContinuingEqualStepOrRandom.py -t 48 -m 1 -N 5 -i /work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.zChr -o /work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/

```

generate running scripts
```
txt_qsub = '''#!/bin/bash
#SBATCH --job-name=examl
#SBATCH --partition=32GB
#SBATCH --nodes=10
#SBATCH --ntasks=320
#SBATCH --time=5-00:00:00
#SBATCH --output=0.%j.out
#SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL

module load openmpi/intel/3.1.1


'''
commandline = '''/home2/s185491/p/anaconda3/anaconda520/bin/python3 /home2/s185491/p/xiaolongTools/utils/runExaML_withFastaForLargeFiles.py -i {filename} -T 320 -r {file_ref}
'''


dc_files = {}
dc_ref = {}
dc_files['CDS'] = '/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.CDS.split5.{N}'
dc_files['intron'] = '/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.intron.split5.{N}'
dc_files['whole'] = '/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.wholeGap7.split5.{N}'
dc_files['zChr'] = '/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.zChr.split5.{N}'

for k in dc_files:
    ls_files = [dc_files[k].format(N = i) for i in range(5)]
    file_ref = '/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.mito'
    fout = open('/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/cmds/qsub_examl_'+k,'w')
    fout.write(txt_qsub.format(key=k))
    for filename in ls_files:
        fout.write(commandline.format(filename=filename, file_ref=file_ref))
    fout.close()
```


 ```
txt = ''' cat {keyword}.split5.* >{keyword}.all5
 /home2/s185491/p/anaconda3/anaconda520/bin/sumtrees.py -f 0.00 {keyword}.all5 >{keyword}.all5.sum
 python /home2/s185491/p/xiaolongTools/utils/trees/sumtreesNEXUS2newick.py -i {keyword}.all5.sum
 /home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py {keyword}.all5.sum.nwk
'''

keywords = '''/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/ExaML_result.Calephelis35.CDS
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/ExaML_result.Calephelis35.intron
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/ExaML_result.Calephelis35.wholeGap7
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/ExaML_result.Calephelis35.zChr
'''.split()

for keyword in keywords:
    print(txt.format(keyword=keyword))

 ```
```
/home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py /work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.mito
```

#### zCDS tree

get sequence
```
import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/mapFile')
import mapfileIO
import glob
import numpy as np
import pickle
import pandas as pd
import os

folderBioHPC = '/work/archive/biophysics/Nick_lab/s185491/Celastrina/fq'
folder_fq = '/archive/butterfly/ready_fastq/'
folder_summary = '/home/xcao/w/2018Calephelis/summary/'

file_summary = os.path.join(folder_summary,'20181217Calephelis.xlsx')
df_summary = pd.read_excel(file_summary,sheet_name='Sheet1')
df_summary = df_summary[df_summary["Filter_rawsoni"]==1]
sample_prefix = [e[1:] for e in df_summary['ID']]
print('unique sample prefix', len(set(sample_prefix)))

folder = '/home/xcao/w/2018Calephelis/summary/map.npInt8.0/'
outfile = '/home/xcao/w/2018Calephelis/summary/map.sitesInfo/gapCount.rawsoni35'
outfileHist = outfile + '.hist'
files = glob.glob(folder+'*')
files = [f for f in files if os.path.basename(f).split('_')[0] in sample_prefix]

print('total map files', len(files))

#ls_mapInt8 = mapfileIO.loadMapBinaries(files, threads=8)#multithreading not working. use the slow method
ls_mapInt8 = [mapfileIO.loadMapBinary(f) for f in files]
dc_mapInt8 = {}
for k,v in zip(files,ls_mapInt8):
    k = os.path.basename(k).split('_')[0]
    dc_mapInt8[k] = v

npFilter_z = mapfileIO.loadMapBinary(filename='/home/xcao/w/2018Calephelis/summary/map.sitesInfo/zChrFilter')
npFilter_sites = mapfileIO.loadMapBinary(filename='/home/xcao/w/2018Calephelis/summary/map.sitesInfo/gff.0inter.2intron.6CDS')
npFilter_z.sum() / npFilter_z.shape[0]
npFilter_zCDS = (npFilter_z) & (npFilter_sites == 6)

with open('/home/xcao/w/2018Calephelis/summary/map.sitesInfo/zChrCDSFilter','wb') as f:
    pickle.dump(npFilter_zCDS, f)

outfile = '/home/xcao/w/2018Calephelis/20190101rawsoni/Calephelis35.zChrCDS'
mapfileIO.npInt8stoFastas(dc_mapInt8, threads=32, output=outfile, headerFun=lambda x:x.split('_')[0], npFilter=npFilter_zCDS, npFilterFun=lambda x:x<=7, merged=True)

#mapfileIO.npInt8stoFastas(files,threads=32, output= '/home/xcao/w/2018Calephelis/20190101rawsoni/Calephelis35.zChrCDS', headerFun=lambda x:x.split('_')[0], npFilter='/home/xcao/w/2018Calephelis/summary/map.sitesInfo/zChrCDSFilter',merged=True)
```

run RAxML
```
scp /home/xcao/w/2018Calephelis/20190101rawsoni/Calephelis35.zChrCDS /work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni
```

```
/home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py /work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/RAxML_bipartitions.Calephelis35.zChrCDS

```

### gene tree simple 35 C. rawsoni

#### get gene trees by simply dividing the sequences to 50kb window

```
python /home2/s185491/p/xiaolongTools/utils/splitFastaAlignments2NPartsContinuingEqualStepOrRandom.py -i /work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.wholeGap7 -o /work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/wholeGap7/ -b 25000 -m 0 -t 48 -w 50000

python /home2/s185491/p/xiaolongTools/utils/splitFastaAlignments2NPartsContinuingEqualStepOrRandom.py -i /work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.CDS -o /work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/CDS/ -b 25000 -m 0 -t 48 -w 50000

python /home2/s185491/p/xiaolongTools/utils/splitFastaAlignments2NPartsContinuingEqualStepOrRandom.py -i /work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.intron -o /work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/intron/ -b 25000 -m 0 -t 48 -w 50000

python /home2/s185491/p/xiaolongTools/utils/splitFastaAlignments2NPartsContinuingEqualStepOrRandom.py -i /work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.zChr -o /work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/zChr/ -b 25000 -m 0 -t 48 -w 50000

```

#### generate running script
```
folders = '''/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/CDS/
/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/intron/
/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/wholeGap7/
/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/zChr/
'''.split()
folder_cmds = '/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/cmds/'

import glob
import os

files = []
for f in folders:
    files += glob.glob(f+'*')
print(len(files))

qusb_txt='''#!/bin/bash
#SBATCH --job-name=raxml
#SBATCH --partition=32GB
#SBATCH --nodes=1
#SBATCH --ntasks=32
#SBATCH --time=5-00:00:00
#SBATCH --output=0.%j.out

python /home2/s185491/p/xiaolongTools/utils/runRAxMLForIndividualSmallFiles.py -i {file_in} -T 32 -N 50

'''
for f in files:
    open(os.path.join(folder_cmds,os.path.basename(f)), 'w').write(qusb_txt.format(file_in=f))

file_cmds = glob.glob(folder_cmds+'*')
open(folder_cmds[:-1]+'.all','w').write('\n'.join(file_cmds))
```

submit the jobs
```
 nohup python /home2/s185491/p/xiaolongTools/utils/submitJobs.py -i /work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/cmds.all -N 32 -t 20 -k raxml &
```

#### sumarize trees

combine outputs
```
cat /work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/CDS.RAxMLbestTree/* >/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/CDS.RAxMLbestTree.all
cat /work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/intron.RAxMLbestTree/* >/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/intron.RAxMLbestTree.all
cat /work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/wholeGap7.RAxMLbestTree/* >/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/wholeGap7.RAxMLbestTree.all
cat /work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/zChr.RAxMLbestTree/* >/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/zChr.RAxMLbestTree.all
```

#### sumtrees
```
txt = '''
/home2/s185491/p/anaconda3/anaconda520/bin/sumtrees.py --ignore-missing-support -f 0.00 {keyword} >{keyword}.sum
python /home2/s185491/p/xiaolongTools/utils/trees/sumtreesNEXUS2newick.py -i {keyword}.sum
/home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py {keyword}.sum.nwk
'''

keywords = '''/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/CDS.RAxMLbestTree.all
/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/intron.RAxMLbestTree.all
/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/wholeGap7.RAxMLbestTree.all
/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/zChr.RAxMLbestTree.all

'''.split()
for keyword in keywords:
    print(txt.format(keyword=keyword))
```

#### Astral
```

txt = '''java -Xmx3000m -jar /home2/s185491/p/Astral/astral.5.6.2.jar -i {f} -o {f}.astral '''

files = '''/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/CDS.RAxMLbestTree.all
/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/intron.RAxMLbestTree.all
/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/wholeGap7.RAxMLbestTree.all
/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/zChr.RAxMLbestTree.all'''.split()

for f in files:
    print(txt.format(f=f))

files = '''/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/CDS.RAxMLbestTree.all.astral
/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/intron.RAxMLbestTree.all.astral
/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/wholeGap7.RAxMLbestTree.all.astral
/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/zChr.RAxMLbestTree.all.astral
'''.split()
for f in files:
    print('/home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py  ' + f)
```

#### densiTree
```

filename = r"C:\P\DensiTree\CDS.RAxMLbestTree.all.rescale2"
filename = r"C:\P\DensiTree\wholeGap7.RAxMLbestTree.all.rescale2"

from ete3 import Tree
lines = open(filename).readlines()
ls_trees = [Tree(e.strip()) for e in lines]
ls_trees.sort(key=lambda x:len(x.get_leaf_names()), reverse=True)
fout = open(filename,'w')
for t in ls_trees:
    fout.write(t.write()+'\n')
fout.close()

file_talbe = r"C:\Users\ATPs\OneDrive\Lab\UTSW\2018Calephelis\phylo\35rawsoni_gene_trees\densitree\nameTable.txt"

dc_name = {}
for line in open(file_talbe):
    es = line.strip().replace(' ','_').split('\t')
    dc_name[es[0]] = '_'.join(es[1:])
    
filename = r"C:\Users\ATPs\OneDrive\Lab\UTSW\2018Calephelis\phylo\35rawsoni_gene_trees\densitree\CDS.svg"
filename = r"C:\Users\ATPs\OneDrive\Lab\UTSW\2018Calephelis\phylo\35rawsoni_gene_trees\densitree\whole.svg"
txt = open(filename).read()
for k,v in dc_name.items():
    txt = txt.replace(k+'</text>\n', v+'</text>\n')
open(filename,'w').write(txt)
```

#### gene trees find subtrees that group rawsoni, freemani, arizonensis together

transfer trees from bioHPC to alea
```
scp -r s185491@nucleus.biohpc.swmed.edu:/work/biophysics/s185491/20181226Calephelis/Phylo/20190103rawsoniGeneTree/wholeGap7.RAxMLbestTree/  /home/xcao/w/2018Calephelis/20190101rawsoni/wholeGenomeGeneTrees
```

get the trees
```
from ete3 import Tree
from multiprocessing import Pool
import pandas as pd
import numpy as np


#filename = r"C:\Users\K\Downloads\chr10_871"
#tree = Tree(open(filename).read())

def addNodeNames(tree):
    '''
    the internal nodes have no names. add unique names like node1, node2 to them
    '''
    n = 1
    for node in tree.traverse():
        if not node.is_leaf():
            node.name = 'node' + str(n)
            n += 1
    return None

def add_species(tree):
    '''
    add speces based on name for leaves
    '''
    for leaf in tree.iter_leaves():
        if leaf.name in dc_sample2species:
            leaf.add_features(species=dc_sample2species[leaf.name])
    return None

def get_leaf_species(tree):
    '''
    get the species names for a tree node
    '''
    return([e.species for e in tree.get_leaves()])

def checkIfSpeciesMono(tree, species):
    '''
    given a tree, leaf belong to species form a monotree
    '''
    tree = tree.copy()
    tree.unroot()
    leaves_test = []
    for leaf in tree.iter_leaves():
        if leaf.species == species:
            leaves_test.append(leaf.name)
    
    for node in tree.iter_descendants():
        tree2 = tree.copy()
        tree2.set_outgroup(node.name)
        test_root = tree.get_common_ancestor(*leaves_test)
        species_identified = get_leaf_species(test_root)
        species_identified = [e for e in species_identified if '?' not in e] #ignore species with '?'
        if len(species_identified) == len(leaves_test):
            return True
    return False

def processTXT(txt):
    tree_id, tree_txt = txt.split()
    tree = Tree(tree_txt)
    addNodeNames(tree)
    add_species(tree)
    tree.unroot()
    addNodeNames(tree)
    dc ={}
    dc['tree_id']=tree_id
    dc['tree'] = tree
    dc['tree_txt'] = tree_txt
    species_checking = ['Calephelis arizonensis','Calephelis freemani','Calephelis nuevoleon','Calephelis rawsoni']
    for s in species_checking:
        dc[s] = checkIfSpeciesMono(tree, s)
    return dc


# prepare samples and species names
file_sample2species = '/home/xcao/w/2018Calephelis/20190101rawsoni/wholeGenomeGeneTrees/id2species'

dc_sample2species = {}
for line in open(file_sample2species):
    line = line.strip()
    k,v = line.split('\t')
    dc_sample2species[k] = v

#remove outgroup in dc_sample2species
testing_groups = ['arizonensis', 'freemani','nuevoleon','rawsoni']
samples_keep = []
for k,v in dc_sample2species.items():
    keep = False
    for e in testing_groups:
        if e in v:
            keep = True
            break
    if keep:
        samples_keep.append(k)
print(len(samples_keep),'samples keep in testing_groups')
dc_sample2species = {k:v for k,v in dc_sample2species.items() if k in samples_keep}

#prepare trees
folder = '/home/xcao/w/2018Calephelis/20190101rawsoni/wholeGenomeGeneTrees/wholeGap7.RAxMLbestTree/'
import glob
files = glob.glob(folder+'*')
print('files of gene trees', len(files))
files.sort(key=lambda x:int(x.split('.')[-1]))
#combine the trees
fout = open(folder.rstrip('/')+'.all', 'w')
for f in files:
    n = f.split('.')[-1]
    t = open(f).read().strip()
    fout.write(n+'\t'+t+'\n')
fout.close()

#remove 
filename = '/home/xcao/w/2018Calephelis/20190101rawsoni/wholeGenomeGeneTrees/wholeGap7.RAxMLbestTree.all'
fileout = filename+'.26'
fout = open(fileout,'w')

def removeNodes(t,dc_sample2species):
    '''
    t is a tree in newick format
    return a tree in newick format
    only include nodes that exist in dc_sample2species
    '''
    tree = Tree(t)
    for l in tree.iter_leaves():
        if l.name not in dc_sample2species:
            l.delete()
    return tree.write()
    
for line in open(filename).readlines():
    n,t = line.split()
    new_t = removeNodes(t,dc_sample2species)
    fout.write(n+'\t'+new_t+'\n')
fout.close()

filename = '/home/xcao/w/2018Calephelis/20190101rawsoni/wholeGenomeGeneTrees/wholeGap7.RAxMLbestTree.all.26'
txts = open(filename).readlines()
pool = Pool(32)
df_tree = pd.DataFrame(pool.map(processTXT,txts))
pool.close()

species_checking = ['Calephelis arizonensis','Calephelis freemani','Calephelis nuevoleon','Calephelis rawsoni']
for species in species_checking:
    print('trees with samples in one monotree for',species, sum(df_tree[species]))

#get trees that separate all four species. No tree
df_good = df_tree[df_tree['Calephelis arizonensis'] & df_tree['Calephelis freemani'] & df_tree['Calephelis nuevoleon'] & df_tree['Calephelis rawsoni']]

outfolder = '/home/xcao/w/2018Calephelis/20190101rawsoni/wholeGenomeGeneTrees/wholeGap7.RAxML.26.keep/'
for n,row in df_good.iterrows():
    open(outfolder+row['tree_id'],'w').write(row['tree_txt'])
    
    
#get trees that separate all four species. No tree
df_good = df_tree[df_tree['Calephelis arizonensis'] & df_tree['Calephelis freemani'] & df_tree['Calephelis nuevoleon'] & df_tree['Calephelis rawsoni']]

outfolder = '/home/xcao/w/2018Calephelis/20190101rawsoni/wholeGenomeGeneTrees/wholeGap7.RAxML.26.keep/'
for n,row in df_good.iterrows():
    tree=Tree(row['tree_txt'])
    tree.unroot()
    tree_txt = tree.write()
    open(outfolder+row['tree_id'],'w').write(tree_txt)
    
```

plot
```
library(ggtree)
library(ggrepel)
library(ggplot2)

tree <- read.tree('/home/xcao/w/2018Calephelis/20190101rawsoni/wholeGenomeGeneTrees/wholeGap7.RAxML.26.keep/52')

xdf.sample2species = read.csv('/home/xcao/w/2018Calephelis/20190101rawsoni/wholeGenomeGeneTrees/id2species',sep = '\t',stringsAsFactors = FALSE,header = FALSE,colClasses = 'character')
colnames(xdf.sample2species) <- c('sample','species')

p <- ggtree(tree,layout = 'daylight') 
p %<+% xdf.sample2species + geom_tiplab2(aes(angle=angle),hjust = 0.5,vjust=2) + geom_tippoint(aes(size=5,color=species,shape=species),alpha=0.7) + scale_shape_manual(values=c(15,16,17,15,16,17))+ theme(legend.position="right")+ guides(colour = guide_legend(override.aes = list(size=5))) + ggtitle('test')+geom_nodelab(size = 5, col= "red")

folder <- '/home/xcao/w/2018Calephelis/20190101rawsoni/wholeGenomeGeneTrees/wholeGap7.RAxML.26.keep/'
files = c('52', '66', '2101')

pdf('/home/xcao/w/2018Calephelis/20190101rawsoni/wholeGenomeGeneTrees/wholeGap7.RAxML.26.4SpeciesTogehter.pdf',width = 15, height = 9)
for (t in files){
  tree <- paste0(folder,t)
  tree <- read.tree(tree)
  p <- ggtree(tree,layout = 'daylight') 
  p <- p %<+% xdf.sample2species + geom_tiplab2(aes(angle=angle),hjust = 0.5,vjust=2) + geom_tippoint(aes(size=5,color=species,shape=species),alpha=0.7) + scale_shape_manual(values=c(15,16,17,15,16,17))+ theme(legend.position="right")+ guides(colour = guide_legend(override.aes = list(size=5))) + ggtitle(t) + geom_nodelab(size = 5, col= "red")
  print(p)
}
dev.off()


```

trees with samples in one monotree for Calephelis arizonensis 2893  
trees with samples in one monotree for Calephelis freemani 51  
trees with samples in one monotree for Calephelis nuevoleon 145  
trees with samples in one monotree for Calephelis rawsoni 1324  


## PCA

### run PCA for 266 samples, CDS, whole, Z and intron

don't forget to run the commands below
```
export PATH=$PATH:/home2/wli/local/EIG6.1.1/bin;
module add gsl/1.15;
module add openblas/intel/0.2.14;
```

generate running scripts
```
import os
file_fastas = '''/work/biophysics/s185491/20181226Calephelis/Phylo/CDS/Calephelis266.CDS
/work/biophysics/s185491/20181226Calephelis/Phylo/intron/Calephelis266.intron.Gap100
/work/biophysics/s185491/20181226Calephelis/Phylo/whole/Calephelis266.wholeGap60
/work/biophysics/s185491/20181226Calephelis/Phylo/zChr/Calephelis266.zChr'''.split()
folder_PCA = '/work/biophysics/s185491/20181226Calephelis/PCA/266/'

txt = '''cd {folder_PCA}  &&  python /home2/s185491/p/xiaolongTools/WenlinTools.Python3/scripts/fasta2EIGinput.py {file_fasta}  &&  /home2/wli/local/EIG6.1.1/bin/smartpca.pl -i {filename}.geno -a {filename}.snp -b {filename}.ind -o {filename}.pca -p {filename}.plot -e {filename}.eval -l {filename}.log -snpweightoutname {filename}.snp_weights.txt &
'''
for file_fasta in file_fastas:
    print(txt.format(folder_PCA=folder_PCA, file_fasta=file_fasta, filename=os.path.basename(file_fasta)))
```

#### plot PCA
```

filename <- 'C:\\Users\\ATPs\\OneDrive\\Lab\\UTSW\\2018Calephelis\\summary\\20181217Calephelis.xlsx'

library(ggplot2)
library(readxl)
library(ggrepel)

mydata <- read_excel(filename,sheet = 'Sheet1')
taxon_table <- table(mydata$`Taxon name`)
taxon_table[taxon_table>5]
p <- c(0:25, 35,36,37,38,43,48:57,64:127)
taxon_shape <- p[1:length(taxon_table)]
names(taxon_shape) <- names(taxon_table)
rename_taxon <- function(x,N=5){
  #x is a taxon name. if count of x >= N, return x, else return 'other'
  if (taxon_table[x]>=N) return(x)
  else return("other")
}
mydata$taxon <- sapply(mydata$`Taxon name`,rename_taxon)
mydata$taxon.shape <- taxon_shape[mydata$`Taxon name`]

ls_PCAs <- list(
  c("PC1_266_zChr", "PC2_266_zChr", "PC3_266_zChr"),
  c("PC1_266_whole","PC2_266_whole", "PC3_266_whole"),
  c("PC1_266_intron", "PC2_266_intron", "PC3_266_intron"),
  c("PC1_266_CDS", "PC2_266_CDS", "PC3_266_CDS")
)

pdf('C:\\Users\\ATPs\\OneDrive\\Lab\\UTSW\\2018Calephelis\\PCA\\Calephelis266.zChr.whole.intron.CDS.pca.pdf',width = 36, height = 19)
for (PCAs in ls_PCAs) {
  p1 <- ggplot(mydata, aes_string(x=PCAs[1],y=PCAs[2])) + geom_point(aes(color=`Taxon name`, size = 5,shape=taxon.shape))  + geom_label_repel(aes(label=`DNA number`),label.size=NA,fill=NA) +scale_shape_identity() 
  p2 <- ggplot(mydata, aes_string(x=PCAs[1],y=PCAs[3])) + geom_point(aes(color=`Taxon name`, size = 5,shape=taxon.shape))  + geom_label_repel(aes(label=`DNA number`),label.size=NA,fill=NA)  +scale_shape_identity()
  p3 <- ggplot(mydata, aes_string(x=PCAs[2],y=PCAs[3])) + geom_point(aes(color=`Taxon name`, size = 5,shape=taxon.shape))  + geom_label_repel(aes(label=`DNA number`),label.size=NA,fill=NA)  +scale_shape_identity()
  print(p1)
  print(p2)
  print(p3)
}

dev.off()

pdf('C:\\Users\\ATPs\\OneDrive\\Lab\\UTSW\\2018Calephelis\\PCA\\Calephelis266.zChr.whole.intron.CDS.pca.noLabel.pdf',width = 36, height = 19)
for (PCAs in ls_PCAs) {
  p1 <- ggplot(mydata, aes_string(x=PCAs[1],y=PCAs[2])) + geom_point(aes(color=`Taxon name`, size = 5,shape=taxon.shape))   +scale_shape_identity() 
  p2 <- ggplot(mydata, aes_string(x=PCAs[1],y=PCAs[3])) + geom_point(aes(color=`Taxon name`, size = 5,shape=taxon.shape))    +scale_shape_identity()
  p3 <- ggplot(mydata, aes_string(x=PCAs[2],y=PCAs[3])) + geom_point(aes(color=`Taxon name`, size = 5,shape=taxon.shape))    +scale_shape_identity()
  print(p1)
  print(p2)
  print(p3)
}

dev.off()

```

### ru PCA for 35 C. rawsoni group

run
```
import os
file_fastas = '''/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.CDS
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.intron
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.mito
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.wholeGap7
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.zChr
'''.split()
folder_PCA = '/work/biophysics/s185491/20181226Calephelis/PCA/20190101rawsoni/'

txt = '''cd {folder_PCA}  &&  python /home2/s185491/p/xiaolongTools/WenlinTools.Python3/scripts/fasta2EIGinput.py {file_fasta}  &&  /home2/wli/local/EIG6.1.1/bin/smartpca.pl -i {filename}.geno -a {filename}.snp -b {filename}.ind -o {filename}.pca -p {filename}.plot -e {filename}.eval -l {filename}.log -snpweightoutname {filename}.snp_weights.txt &
'''
for file_fasta in file_fastas:
    print(txt.format(folder_PCA=folder_PCA, file_fasta=file_fasta, filename=os.path.basename(file_fasta)))
```

plot
```
filename <- 'C:\\Users\\ATPs\\OneDrive\\Lab\\UTSW\\2018Calephelis\\summary\\20181217Calephelis.xlsx'

library(ggplot2)
library(readxl)
library(ggrepel)
library(tidyr)

mydata1 <- read_excel(filename,sheet = 'Sheet1')
taxon_table <- table(mydata1$`Taxon name`)
taxon_table[taxon_table>5]
p <- c(0:25, 35,36,37,38,43,48:57,64:127)
taxon_shape <- p[1:length(taxon_table)]
names(taxon_shape) <- names(taxon_table)
rename_taxon <- function(x,N=5){
  #x is a taxon name. if count of x >= N, return x, else return 'other'
  if (taxon_table[x]>=N) return(x)
  else return("other")
}
mydata1$taxon <- sapply(mydata1$`new_Taxon_name`,rename_taxon)
mydata1$taxon.shape <- taxon_shape[mydata1$`new_Taxon_name`]

PCA_head.str <- "PC1_35_zChr	PC2_35_zChr	PC3_35_zChr	PC1_35_whole	PC2_35_whole	PC3_35_whole	PC1_35_intron	PC2_35_intron	PC3_35_intron	PC1_35_CDS	PC2_35_CDS	PC3_35_CDS	PC1_35_mito	PC2_35_mito	PC3_35_mito"
PCA_head.vector <- strsplit(PCA_head.str,split = '\t')[[1]]


pdf('C:\\Users\\ATPs\\OneDrive\\Lab\\UTSW\\2018Calephelis\\PCA\\Calephelis35.zChr.whole.intron.CDS.mito.pca.pdf',width = 18, height = 9)
for (n in 1:(length(PCA_head.vector)/3)) {
  PCAs <- PCA_head.vector[(n*3-2):(n*3)]
  mydata <- drop_na(mydata1,PCAs[1])
  p1 <- ggplot(mydata, aes_string(x=PCAs[1],y=PCAs[2])) + geom_point(aes(color=`new_Taxon_name`, size = 5,shape=new_Taxon_name))  + geom_label_repel(aes(label=`DNA number`),label.size=NA,fill=NA) + scale_shape_manual(values = rep(15:19, 6)) 
  p2 <- ggplot(mydata, aes_string(x=PCAs[1],y=PCAs[3])) + geom_point(aes(color=`new_Taxon_name`, size = 5,shape=new_Taxon_name))  + geom_label_repel(aes(label=`DNA number`),label.size=NA,fill=NA)  + scale_shape_manual(values = rep(15:19, 6))
  p3 <- ggplot(mydata, aes_string(x=PCAs[2],y=PCAs[3])) + geom_point(aes(color=`new_Taxon_name`, size = 5,shape=new_Taxon_name))  + geom_label_repel(aes(label=`DNA number`),label.size=NA,fill=NA)  + scale_shape_manual(values = rep(15:19, 6))
  figures = list(p1,p2,p3)
  for (p in figures) print(p + guides(color = guide_legend(override.aes = list(size=5))))
}

dev.off()

```

### PCA 35 rawsoni with no outgroup

```
import os
from Bio import SeqIO
file_fastas = '''/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.CDS
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.intron
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.mito
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.wholeGap7
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.zChr
'''.split()
txt = '''cd {folder_PCA}  &&  python /home2/s185491/p/xiaolongTools/WenlinTools.Python3/scripts/fasta2EIGinput.py {file_fasta}  &&  /home2/wli/local/EIG6.1.1/bin/smartpca.pl -i {filename}.geno -a {filename}.snp -b {filename}.ind -o {filename}.pca -p {filename}.plot -e {filename}.eval -l {filename}.log -snpweightoutname {filename}.snp_weights.txt &
'''

folder_PCA = '/work/biophysics/s185491/20181226Calephelis/PCA/20190106rawsoni_noOutgroup/'
outgroups = ['5951', '18033G04', '15098B05', '16101C08', '6994', '5254', '7144', '16101B02', '3505']
for file_fasta in file_fastas:
    basename = os.path.basename(file_fasta)
    outfile = os.path.join(folder_PCA, basename.replace('Calephelis35', 'Calephelis26'))
    fout = open(outfile,'w')
    for s in SeqIO.parse(file_fasta,'fasta'):
        if s.id not in outgroups:
            fout.write('>'+s.id+'\n'+str(s.seq)+'\n')
    fout.close()
    os.system(txt.format(folder_PCA=folder_PCA, file_fasta=outfile, filename=os.path.basename(outfile)))


```


```
filename <- 'C:\\Users\\ATPs\\OneDrive\\Lab\\UTSW\\2018Calephelis\\summary\\20181217Calephelis.xlsx'

library(ggplot2)
library(readxl)
library(ggrepel)
library(tidyr)

mydata1 <- read_excel(filename,sheet = 'Sheet1')
taxon_table <- table(mydata1$`Taxon name`)
taxon_table[taxon_table>5]
p <- c(0:25, 35,36,37,38,43,48:57,64:127)
taxon_shape <- p[1:length(taxon_table)]
names(taxon_shape) <- names(taxon_table)
rename_taxon <- function(x,N=5){
  #x is a taxon name. if count of x >= N, return x, else return 'other'
  if (taxon_table[x]>=N) return(x)
  else return("other")
}
mydata1$taxon <- sapply(mydata1$`new_Taxon_name`,rename_taxon)
mydata1$taxon.shape <- taxon_shape[mydata1$`new_Taxon_name`]

PCA_head.str <- "PC1_26_zChr	PC2_26_zChr	PC3_26_zChr	PC1_26_whole	PC2_26_whole	PC3_26_whole	PC1_26_intron	PC2_26_intron	PC3_26_intron	PC1_26_CDS	PC2_26_CDS	PC3_26_CDS	PC1_26_mito	PC2_26_mito	PC3_26_mito
"
PCA_head.vector <- strsplit(PCA_head.str,split = '\t')[[1]]


pdf('C:\\Users\\ATPs\\OneDrive\\Lab\\UTSW\\2018Calephelis\\PCA\\Calephelis26.zChr.whole.intron.CDS.mito.pca.pdf',width = 18, height = 9)
for (n in 1:(length(PCA_head.vector)/3)) {
  PCAs <- PCA_head.vector[(n*3-2):(n*3)]
  mydata <- drop_na(mydata1,PCAs[1])
  p1 <- ggplot(mydata, aes_string(x=PCAs[1],y=PCAs[2])) + geom_point(aes(color=`new_Taxon_name`, size = 5,shape=new_Taxon_name))  + geom_label_repel(aes(label=`DNA number`),label.size=NA,fill=NA) + scale_shape_manual(values = rep(15:19, 6)) 
  p2 <- ggplot(mydata, aes_string(x=PCAs[1],y=PCAs[3])) + geom_point(aes(color=`new_Taxon_name`, size = 5,shape=new_Taxon_name))  + geom_label_repel(aes(label=`DNA number`),label.size=NA,fill=NA)  + scale_shape_manual(values = rep(15:19, 6))
  p3 <- ggplot(mydata, aes_string(x=PCAs[2],y=PCAs[3])) + geom_point(aes(color=`new_Taxon_name`, size = 5,shape=new_Taxon_name))  + geom_label_repel(aes(label=`DNA number`),label.size=NA,fill=NA)  + scale_shape_manual(values = rep(15:19, 6))
  figures = list(p1,p2,p3)
  for (p in figures) print(p + guides(color = guide_legend(override.aes = list(size=5))))
}

dev.off()
```

## Structure

### 266 sequences test

#### convert sequences to structure input

```
files = '''
/work/biophysics/s185491/20181226Calephelis/Phylo/CDS/Calephelis266.CDS.split5.0
/work/biophysics/s185491/20181226Calephelis/Phylo/CDS/Calephelis266.CDS.split5.1
/work/biophysics/s185491/20181226Calephelis/Phylo/CDS/Calephelis266.CDS.split5.2
/work/biophysics/s185491/20181226Calephelis/Phylo/CDS/Calephelis266.CDS.split5.3
/work/biophysics/s185491/20181226Calephelis/Phylo/CDS/Calephelis266.CDS.split5.4
/work/biophysics/s185491/20181226Calephelis/Phylo/intron/Calephelis266.intron.Gap100.split5.0
/work/biophysics/s185491/20181226Calephelis/Phylo/intron/Calephelis266.intron.Gap100.split5.1
/work/biophysics/s185491/20181226Calephelis/Phylo/intron/Calephelis266.intron.Gap100.split5.2
/work/biophysics/s185491/20181226Calephelis/Phylo/intron/Calephelis266.intron.Gap100.split5.3
/work/biophysics/s185491/20181226Calephelis/Phylo/intron/Calephelis266.intron.Gap100.split5.4
/work/biophysics/s185491/20181226Calephelis/Phylo/mito/Calephelis266.mito
/work/biophysics/s185491/20181226Calephelis/Phylo/whole/Calephelis266.wholeGap60.split5.0
/work/biophysics/s185491/20181226Calephelis/Phylo/whole/Calephelis266.wholeGap60.split5.1
/work/biophysics/s185491/20181226Calephelis/Phylo/whole/Calephelis266.wholeGap60.split5.2
/work/biophysics/s185491/20181226Calephelis/Phylo/whole/Calephelis266.wholeGap60.split5.3
/work/biophysics/s185491/20181226Calephelis/Phylo/whole/Calephelis266.wholeGap60.split5.4
/work/biophysics/s185491/20181226Calephelis/Phylo/zChr/Calephelis266.zChr.split5.0
/work/biophysics/s185491/20181226Calephelis/Phylo/zChr/Calephelis266.zChr.split5.1
/work/biophysics/s185491/20181226Calephelis/Phylo/zChr/Calephelis266.zChr.split5.2
/work/biophysics/s185491/20181226Calephelis/Phylo/zChr/Calephelis266.zChr.split5.3
/work/biophysics/s185491/20181226Calephelis/Phylo/zChr/Calephelis266.zChr.split5.4
'''.split()


commandline = 'python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/fasta2STRUCTUREinput_haploid_nt.py -i {file_in} -o {file_out} -t 56 -g {gap} -N 4 -H True'
outfolder = '/work/biophysics/s185491/20181226Calephelis/structure/266/'
import os

for file in files:
    if 'CDS.split' in file:
        gap = 0.8
    elif 'intron.Gap' in file:
        gap = 0.85
    elif 'wholeGap60.split' in file:
        gap = 0.9
    elif 'zChr.split' in file:
        gap = 0.8
    elif 'Calephelis266.mito' in file:
        gap = 0.5
    print(commandline.format(file_in = file,gap=gap, file_out = os.path.join(outfolder, os.path.basename(file))))
    
    
```

#### run Structure

```
files = '''
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.CDS.split5.0
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.CDS.split5.1
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.CDS.split5.2
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.CDS.split5.3
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.CDS.split5.4
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.intron.Gap100.split5.0
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.intron.Gap100.split5.1
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.intron.Gap100.split5.2
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.intron.Gap100.split5.3
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.intron.Gap100.split5.4
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.mito
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.wholeGap60.split5.0
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.wholeGap60.split5.1
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.wholeGap60.split5.2
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.wholeGap60.split5.3
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.wholeGap60.split5.4
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.zChr.split5.0
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.zChr.split5.1
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.zChr.split5.2
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.zChr.split5.3
/work/biophysics/s185491/20181226Calephelis/structure/266/Calephelis266.zChr.split5.4
'''.split()

snps = '''
32468
32057
32230
32084
31706
62363
62500
62052
62581
62456
3731
74496
74293
74093
73866
74081
23622
23627
23443
23684
23509
'''.split()

commandline = 'python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/runSTRUCTUREwithSTRUCTUREinput.py -i {file_in} -I 266 -L {snp_len} -s 1000 -P 2,3,4,5,6,7,8,9,10,11,12,13,14,15 -o /work/biophysics/s185491/20181226Calephelis/structure/266/cmds/jobs'
outfolder = '/work/biophysics/s185491/20181226Calephelis/structure/266/'

for file_in, snp_len in zip(files,snps):
    print(commandline.format(file_in=file_in, snp_len=snp_len))
```

python /home2/s185491/p/xiaolongTools/utils/splitFiles2Nparts.py -i /work/biophysics/s185491/20181226Calephelis/structure/266/cmds/jobs -N 6

#### collect data

convert data in two format, one is good for R or python, one is good for CLUMPP
```
import os
import glob
import sys
sys.path.append('/home2/s185491/p/xiaolongTools/utils/STRUCTURE')
import extractStructureTableFromCommandlineOutput
folder_structure = '/work/biophysics/s185491/20181226Calephelis/structure/266/'
outfolder = '/work/biophysics/s185491/20181226Calephelis/structure/266/results/'
files_structure = glob.glob(folder_structure +'runSTRUCTURE*/*_f')
for f in files_structure:
    key = os.path.basename(f)
    outfile = os.path.join(outfolder, key)

outfolder= '/work/biophysics/s185491/20181226Calephelis/structure/266/results2/'
dc_results = {}
for f in files_structure:
    key = os.path.basename(f)
    outfile = os.path.join(outfolder, key)
    dc_results[key] = extractStructureTableFromCommandlineOutput.extractStructureTableFromCommandlineOutput(filename=f, output=outfile, processLineForR=0)
```

#### plot

```
library(readxl)
library(tidyr)
library(dplyr)
library(ggplot2)
library(gridExtra)

folder_files <- '/home/xcao/w/2018Calephelis/20181231Structure/results'
result_files <- list.files(folder_files,full.names = FALSE)
basetypes <- c("intron.Gap100","CDS","mito","wholeGap60","zChr")

f.getbasetype <- function(x){
  for (keyword in basetypes){
    if (grepl(keyword,x)) return(keyword)
  }
}

f.getSplitgroup <- function(x){
  if (grepl('.split',x)) {
    return(as.integer(gsub("(.*split\\d\\.)|(\\.STRUCTUREresult.*)","",x)))
  } else return(0)
}

f.getRandint <- function(x){
  #x="Calephelis266.zChr.split5.4.STRUCTUREresult.s1000.pop8_f" return 1000
  return(as.integer(gsub('(.*STRUCTUREresult.s)|(.pop.*_f)','',x)))
}

f.getPopulation <- function(x) {
  return(as.integer(gsub('(.*STRUCTUREresult.s\\d*.pop)|(_f)','',x)))
}

df_files <- data.frame(baseType=sapply(result_files, f.getbasetype),
                       splitID=sapply(result_files, f.getSplitgroup),
                       seed=sapply(result_files, f.getRandint),
                       population=sapply(result_files,f.getPopulation),
                       filename=file.path(folder_files,result_files), stringsAsFactors = FALSE)

df_files <- df_files[order(df_files$baseType, df_files$population,df_files$seed,df_files$splitID),]

F.renameHead <- function(x){
  x[1] = 'sample'
  x[2:length(x)] = paste0('pop',1:(length(x)-1))
  return(x)
}

filename2 <- '/home/xcao/w/2018Calephelis/summary/20181217Calephelis.xlsx'
xdf_info <- read_excel(filename2,sheet = 'Sheet1')
xdf_info <- xdf_info[!is.na(xdf_info$order_266mitoTree),]
# v_id2Treename <- xdf_info$Treename
# names(v_id2Treename) <- sapply(xdf_info$ID, function(x)substring(x,2))
v_id2species <- xdf_info$new_Taxon_name
names(v_id2species) <- sapply(xdf_info$ID, function(x)substring(x,2))
species_in_order <- xdf_info[order(xdf_info$order_266mitoTree),]$ID
species_in_order <- substr(species_in_order,2,length(species_in_order))


oneline = df_files[1,]
xdf_s <- read.csv(oneline$filename,header = FALSE,sep = '\t', stringsAsFactors = FALSE)
colnames(xdf_s) <- F.renameHead(colnames(xdf_s))
xdf_s$species <- v_id2species[xdf_s$sample]
#xdf_s$treename <- v_id2Treename[xdf_s$sample]
xdf_s.clear <- gather(xdf_s,key = 'pop',value = 'fraction',-sample,-species)
xdf_s.clear <- xdf_s.clear[order(xdf_s.clear$species,xdf_s.clear$pop,-xdf_s.clear$fraction),]

p <- ggplot(xdf_s.clear,aes(x=sample,y=fraction,fill=pop)) + geom_bar(stat = 'identity') + scale_x_discrete(limits=species_in_order)



F.plotOneline <- function(oneline,xtext=FALSE){
  xdf_s <- read.csv(oneline$filename,header = FALSE,sep = '\t', stringsAsFactors = FALSE)
  colnames(xdf_s) <- F.renameHead(colnames(xdf_s))
  xdf_s$species <- v_id2species[xdf_s$sample]
  #xdf_s$treename <- v_id2Treename[xdf_s$sample]
  xdf_s.clear <- gather(xdf_s,key = 'pop',value = 'fraction',-sample,-species)
  xdf_s.clear <- xdf_s.clear[order(xdf_s.clear$species,xdf_s.clear$pop,-xdf_s.clear$fraction),]
  
  p <- ggplot(xdf_s.clear,aes(x=sample,y=fraction,fill=pop)) + geom_bar(stat = 'identity') + scale_x_discrete(limits=species_in_order) + ylab(paste0('seed',oneline$seed,'_',oneline$baseType,'_K=',oneline$population," ",oneline$splitID))
  if (xtext) {
    p <- p + theme(legend.position="none", axis.text.x= element_text(angle = 90, hjust = 1), axis.title.x = element_blank())
  }
  else {
    p <-  p + theme(legend.position="none", axis.text.x= element_blank(), axis.title.x = element_blank())
    }
 return(p)
}

F.plotMultipleLines <- function(filelines,xtext=FALSE){
  p <- list()
  for (i in 1:nrow(filelines)){
    p[[i]] <- F.plotOneline(filelines[i,],xtext = xtext)
  }
  #p[[i+1]] <- F.plotOneline(filelines[i+1,], xtext = TRUE)
  return(grid.arrange(grobs=p,ncol=1))
}

filelines = df_files[df_files$baseType == 'mito',]
F.plotMultipleLines(filelines = filelines)

filelines = df_files[df_files$baseType == 'mito',]
F.plotMultipleLines(filelines = filelines,xtext = TRUE)

filelines = df_files[df_files$baseType == 'zChr',]
F.plotMultipleLines(filelines = filelines)

filelines = df_files[df_files$baseType == 'CDS',]
F.plotMultipleLines(filelines = filelines)

filelines = df_files[df_files$baseType == 'CDS' &df_files$splitID==0,]
F.plotMultipleLines(filelines = filelines,xtext = TRUE)


filelines = df_files[df_files$baseType == 'wholeGap60',]
F.plotMultipleLines(filelines = filelines)

filelines = df_files[df_files$baseType == 'intron.Gap100',]
F.plotMultipleLines(filelines = filelines)


```

### 35 rawsoni

#### convert sequence to structure input
```
files = '''
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.CDS
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.intron
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.mito
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.wholeGap7
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.zChr
'''.split()


commandline = 'python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/fasta2STRUCTUREinput_haploid_nt.py -i {file_in} -o {file_out} -t 56 -g {gap} -N 4 -H True'
outfolder = '/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/'
import os

for file in files:
    if 'CDS' in file:
        gap = 0.8
    elif 'intron' in file:
        gap = 0.85
    elif 'whole' in file:
        gap = 0.9
    elif 'zChr' in file:
        gap = 0.8
    elif 'mito' in file:
        gap = 0.5
    print(commandline.format(file_in = file,gap=gap, file_out = os.path.join(outfolder, os.path.basename(file))))
```

#### run structure
```
files = '''/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/Calephelis35.CDS
/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/Calephelis35.intron
/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/Calephelis35.mito
/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/Calephelis35.wholeGap7
/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/Calephelis35.zChr
'''.split()

snps = '''
168647
529582
2196
780165
109709
'''.split()

commandline = 'python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/runSTRUCTUREwithSTRUCTUREinput.py -i {file_in} -I 35 -L {snp_len} -s 1000,2000,3000 -P 2,3,4,5,6 -o /work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/jobs'
outfolder = '/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/'
import os
for file_in, snp_len in zip(files,snps):
    os.system(commandline.format(file_in=file_in, snp_len=snp_len))
```
modify the jobs file and submit

#### collect data
```
import os
import glob
import sys
sys.path.append('/home2/s185491/p/xiaolongTools/utils/STRUCTURE')
import extractStructureTableFromCommandlineOutput
folder_structure = '/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/'
outfolder = '/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/result/'
files_structure = glob.glob(folder_structure +'runSTRUCTURE*/*_f')
for f in files_structure:
    key = os.path.basename(f)
    outfile = os.path.join(outfolder, key)
    extractStructureTableFromCommandlineOutput.extractStructureTableFromCommandlineOutput(filename=f, output=outfile, processLineForR=1)


```

#### plot
```
library(readxl)
library(tidyr)
library(dplyr)
library(ggplot2)
library(gridExtra)

folder_files <- '/home/xcao/w/2018Calephelis/20181231Structure/35/result/'
result_files <- list.files(folder_files,full.names = FALSE)
basetypes <- c("intron","CDS","mito","wholeGap7","zChr")

f.getbasetype <- function(x){
  for (keyword in basetypes){
    if (grepl(keyword,x)) return(keyword)
  }
}

f.getSplitgroup <- function(x){
  if (grepl('.split',x)) {
    return(as.integer(gsub("(.*split\\d\\.)|(\\.STRUCTUREresult.*)","",x)))
  } else return(0)
}

f.getRandint <- function(x){
  #x="Calephelis266.zChr.split5.4.STRUCTUREresult.s1000.pop8_f" return 1000
  return(as.integer(gsub('(.*STRUCTUREresult.s)|(.pop.*_f)','',x)))
}

f.getPopulation <- function(x) {
  return(as.integer(gsub('(.*STRUCTUREresult.s\\d*.pop)|(_f)','',x)))
}

df_files <- data.frame(baseType=sapply(result_files, f.getbasetype),
                       splitID=sapply(result_files, f.getSplitgroup),
                       seed=sapply(result_files, f.getRandint),
                       population=sapply(result_files,f.getPopulation),
                       filename=file.path(folder_files,result_files), stringsAsFactors = FALSE)

df_files <- df_files[order(df_files$baseType, df_files$population,df_files$seed,df_files$splitID),]

F.renameHead <- function(x){
  x[1] = 'sample'
  x[2:length(x)] = paste0('pop',1:(length(x)-1))
  return(x)
}

filename2 <- '/home/xcao/w/2018Calephelis/summary/20181217Calephelis.xlsx'
xdf_info <- read_excel(filename2,sheet = 'Sheet1')
xdf_info <- xdf_info[!is.na(xdf_info$Filter_rawsoni),]
# v_id2Treename <- xdf_info$Treename
# names(v_id2Treename) <- sapply(xdf_info$ID, function(x)substring(x,2))
v_id2species <- xdf_info$new_Taxon_name
names(v_id2species) <- sapply(xdf_info$ID, function(x)substring(x,2))
species_in_order <- xdf_info[order(xdf_info$order_266mitoTree),]$ID
species_in_order <- substr(species_in_order,2,length(species_in_order))


oneline = df_files[1,]
xdf_s <- read.csv(oneline$filename,header = FALSE,sep = '\t', stringsAsFactors = FALSE)
colnames(xdf_s) <- F.renameHead(colnames(xdf_s))
xdf_s$species <- v_id2species[xdf_s$sample]
#xdf_s$treename <- v_id2Treename[xdf_s$sample]
xdf_s.clear <- gather(xdf_s,key = 'pop',value = 'fraction',-sample,-species)
xdf_s.clear <- xdf_s.clear[order(xdf_s.clear$species,xdf_s.clear$pop,-xdf_s.clear$fraction),]

p <- ggplot(xdf_s.clear,aes(x=sample,y=fraction,fill=pop)) + geom_bar(stat = 'identity') + scale_x_discrete(limits=species_in_order)



F.plotOneline <- function(oneline,xtext=FALSE){
  xdf_s <- read.csv(oneline$filename,header = FALSE,sep = '\t', stringsAsFactors = FALSE)
  colnames(xdf_s) <- F.renameHead(colnames(xdf_s))
  xdf_s$species <- v_id2species[xdf_s$sample]
  #xdf_s$treename <- v_id2Treename[xdf_s$sample]
  xdf_s.clear <- gather(xdf_s,key = 'pop',value = 'fraction',-sample,-species)
  xdf_s.clear <- xdf_s.clear[order(xdf_s.clear$species,xdf_s.clear$pop,-xdf_s.clear$fraction),]
  
  p <- ggplot(xdf_s.clear,aes(x=sample,y=fraction,fill=pop)) + geom_bar(stat = 'identity') + scale_x_discrete(limits=species_in_order) + ylab(paste0('seed',oneline$seed,'_',oneline$baseType,'_K=',oneline$population," ",oneline$splitID))
  if (xtext) {
    p <- p + theme(legend.position="none", axis.text.x= element_text(angle = 90, hjust = 1), axis.title.x = element_blank())
  }
  else {
    p <-  p + theme(legend.position="none", axis.text.x= element_blank(), axis.title.x = element_blank())
  }
  return(p)
}

F.plotMultipleLines <- function(filelines,xtext=FALSE){
  p <- list()
  for (i in 1:nrow(filelines)){
    p[[i]] <- F.plotOneline(filelines[i,],xtext = xtext)
  }
  #p[[i+1]] <- F.plotOneline(filelines[i+1,], xtext = TRUE)
  return(grid.arrange(grobs=p,ncol=1))
}

#"intron","CDS","mito","wholeGap7","zChr"
filelines = df_files[df_files$baseType == 'mito'& df_files$seed == 1000,]
F.plotMultipleLines(filelines = filelines,xtext = TRUE)

pdf('/home/xcao/w/2018Calephelis/20181231Structure/35/35rawsoni.pdf',width = 8, height = 12)
filelines = df_files[df_files$baseType == 'mito' & df_files$seed == 1000,]
F.plotMultipleLines(filelines = filelines)



filelines = df_files[df_files$baseType == 'zChr'& df_files$seed == 1000,]
F.plotMultipleLines(filelines = filelines)

filelines = df_files[df_files$baseType == 'CDS'& df_files$seed == 1000,]
F.plotMultipleLines(filelines = filelines)


filelines = df_files[df_files$baseType == 'wholeGap7'& df_files$seed == 1000,]
F.plotMultipleLines(filelines = filelines)

filelines = df_files[df_files$baseType == 'intron'& df_files$seed == 1000,]
F.plotMultipleLines(filelines = filelines)
dev.off()

```

### rawsoni no outgroup

```
files = '''/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/Calephelis35.CDS
/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/Calephelis35.intron
/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/Calephelis35.mito
/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/Calephelis35.wholeGap7
/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/Calephelis35.zChr
'''.split()

snps = '''
168647
529582
2196
780165
109709
'''.split()

filter1 = ['5951', '18033G04', '15098B05', '16101C08', '6994', '5254', '7144', '16101B02', '3505']
filter2 = ['7144', '16101B02', '3505']
outfolder1 = '/work/biophysics/s185491/20181226Calephelis/structure/rawsoni26/'
outfolder2 = '/work/biophysics/s185491/20181226Calephelis/structure/rawsoni32/'

commandline = 'python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/runSTRUCTUREwithSTRUCTUREinput.py -i {file_in} -I {inputN} -L {snp_len} -s 1000,2000,3000 -P 2,3,4,5,6,7,8,9 -o /work/biophysics/s185491/20181226Calephelis/structure/rawsoni_jobs'

import os
for file_in, snp_len in zip(files,snps):
    l = open(file_in).readlines()
    file_out1 = os.path.join(outfolder1,os.path.basename(file_in).replace('Calephelis35','Calephelis26'))
    file_out2 = os.path.join(outfolder2,os.path.basename(file_in).replace('Calephelis35','Calephelis32'))
    fout1 = open(file_out1, 'w')
    fout2 = open(file_out2, 'w')
    for line in l:
        if line.split('\t')[0] not in filter1:
            fout1.write(line)
        if line.split('\t')[0] not in filter2:
            fout2.write(line)
    fout1.close()
    fout2.close()
    os.system(commandline.format(file_in=file_out1, snp_len=snp_len, inputN=26))
    os.system(commandline.format(file_in=file_out2, snp_len=snp_len, inputN=32))


```

#### collect data

```
import os
import glob
import sys
sys.path.append('/home2/s185491/p/xiaolongTools/utils/STRUCTURE')
import extractStructureTableFromCommandlineOutput
folder_structure = '/work/biophysics/s185491/20181226Calephelis/structure/rawsoni26/'
outfolder = '/work/biophysics/s185491/20181226Calephelis/structure/rawsoni26/result/'
files_structure = glob.glob(folder_structure +'runSTRUCTURE*/*_f')
for f in files_structure:
    key = os.path.basename(f)
    outfile = os.path.join(outfolder, key)
    extractStructureTableFromCommandlineOutput.extractStructureTableFromCommandlineOutput(filename=f, output=outfile, processLineForR=1)
```

#### plot

```

```

#### rawsoni 26 zChrCDS
```
files = '''
/work/biophysics/s185491/20181226Calephelis/Phylo/20190101rawsoni/Calephelis35.zChrCDS
'''.split()


commandline = 'python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/fasta2STRUCTUREinput_haploid_nt.py -i {file_in} -o {file_out} -t 56 -g {gap} -N 4 -H True'
outfolder = '/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/'
import os

for file in files:
    if 'CDS' in file:
        gap = 0.8
    elif 'intron' in file:
        gap = 0.85
    elif 'whole' in file:
        gap = 0.9
    elif 'zChr' in file:
        gap = 0.8
    elif 'mito' in file:
        gap = 0.5
    print(commandline.format(file_in = file,gap=gap, file_out = os.path.join(outfolder, os.path.basename(file))))


files = '''/work/biophysics/s185491/20181226Calephelis/structure/35rawsoni/Calephelis35.zChrCDS
'''.split()

snps = '''
3989
'''.split()

filter1 = ['5951', '18033G04', '15098B05', '16101C08', '6994', '5254', '7144', '16101B02', '3505']
outfolder1 = '/work/biophysics/s185491/20181226Calephelis/structure/rawsoni26/'

commandline = 'python /home2/s185491/p/xiaolongTools/utils/STRUCTURE/runSTRUCTUREwithSTRUCTUREinput.py -i {file_in} -I {inputN} -L {snp_len} -s 1000,2000,3000 -P 2,3,4,5,6,7,8,9 -o /work/biophysics/s185491/20181226Calephelis/structure/rawsoni_jobs2'

import os
for file_in, snp_len in zip(files,snps):
    l = open(file_in).readlines()
    file_out1 = os.path.join(outfolder1,os.path.basename(file_in).replace('Calephelis35','Calephelis26'))
    fout1 = open(file_out1, 'w')
    for line in l:
        if line.split('\t')[0] not in filter1:
            fout1.write(line)

    fout1.close()
    os.system(commandline.format(file_in=file_out1, snp_len=snp_len, inputN=26))

```

#### plot 26 rawsoni
```
library(readxl)
library(tidyr)
library(dplyr)
library(ggplot2)
library(gridExtra)

folder_files <- '/home/xcao/w/2018Calephelis/20181231Structure/26/result/'
result_files <- list.files(folder_files,full.names = FALSE)
basetypes <- c("intron","CDS","mito","wholeGap7","zChr","zChrCDS")

f.getbasetype <- function(x){
  return (strsplit(x,split = '\\.')[[1]][2])
}

f.getSplitgroup <- function(x){
  if (grepl('.split',x)) {
    return(as.integer(gsub("(.*split\\d*\\.)|(\\.STRUCTUREresult.*)","",x)))
  } else return(0)
}

f.getRandint <- function(x){
  #x="Calephelis266.zChr.split5.4.STRUCTUREresult.s1000.pop8_f" return 1000
  return(as.integer(gsub('(.*STRUCTUREresult.s)|(.pop.*_f)','',x)))
}

f.getPopulation <- function(x) {
  return(as.integer(gsub('(.*STRUCTUREresult.s\\d*.pop)|(_f)','',x)))
}

df_files <- data.frame(baseType=sapply(result_files, f.getbasetype),
                       splitID=sapply(result_files, f.getSplitgroup),
                       seed=sapply(result_files, f.getRandint),
                       population=sapply(result_files,f.getPopulation),
                       filename=file.path(folder_files,result_files), stringsAsFactors = FALSE)

df_files <- df_files[order(df_files$baseType, df_files$population,df_files$seed,df_files$splitID),]

F.renameHead <- function(x){
  x[1] = 'sample'
  x[2:length(x)] = paste0('pop',1:(length(x)-1))
  return(x)
}

filename2 <- '/home/xcao/w/2018Calephelis/summary/20181217Calephelis.xlsx'
xdf_info <- read_excel(filename2,sheet = 'Sheet1')
xdf_info <- xdf_info[!is.na(xdf_info$Filter_rawsoni),]
# v_id2Treename <- xdf_info$Treename
# names(v_id2Treename) <- sapply(xdf_info$ID, function(x)substring(x,2))
v_id2species <- xdf_info$new_Taxon_name
names(v_id2species) <- sapply(xdf_info$ID, function(x)substring(x,2))
species_in_order <- xdf_info[order(xdf_info$order_266mitoTree),]$ID
species_in_order <- substr(species_in_order,2,length(species_in_order))


oneline = df_files[1,]
xdf_s <- read.csv(oneline$filename,header = FALSE,sep = '\t', stringsAsFactors = FALSE)
colnames(xdf_s) <- F.renameHead(colnames(xdf_s))
xdf_s$species <- v_id2species[xdf_s$sample]
#xdf_s$treename <- v_id2Treename[xdf_s$sample]
xdf_s.clear <- gather(xdf_s,key = 'pop',value = 'fraction',-sample,-species)
xdf_s.clear <- xdf_s.clear[order(xdf_s.clear$species,xdf_s.clear$pop,-xdf_s.clear$fraction),]

p <- ggplot(xdf_s.clear,aes(x=sample,y=fraction,fill=pop)) + geom_bar(stat = 'identity') + scale_x_discrete(limits=species_in_order)



F.plotOneline <- function(oneline,xtext=FALSE){
  xdf_s <- read.csv(oneline$filename,header = FALSE,sep = '\t', stringsAsFactors = FALSE)
  colnames(xdf_s) <- F.renameHead(colnames(xdf_s))
  xdf_s$species <- v_id2species[xdf_s$sample]
  #xdf_s$treename <- v_id2Treename[xdf_s$sample]
  xdf_s.clear <- gather(xdf_s,key = 'pop',value = 'fraction',-sample,-species)
  xdf_s.clear <- xdf_s.clear[order(xdf_s.clear$species,xdf_s.clear$pop,-xdf_s.clear$fraction),]
  
  p <- ggplot(xdf_s.clear,aes(x=sample,y=fraction,fill=pop)) + geom_bar(stat = 'identity') + scale_x_discrete(limits=species_in_order[species_in_order %in% xdf_s$sample]) + ylab(paste0('seed',oneline$seed,'_',oneline$baseType,'_K=',oneline$population," ",oneline$splitID))
  if (xtext) {
    p <- p + geom_text(aes(x=sample,y=0,label=paste(sample,species)),angle=90,hjust=0)+ theme(legend.position="none", axis.text.x= element_blank(), axis.title.x = element_blank())
    #p <- p + theme(legend.position="none", axis.text.x= element_text(angle = 90, hjust = 1), axis.title.x = element_blank())
  }
  else {
    p <-  p + theme(legend.position="none", axis.text.x= element_blank(), axis.title.x = element_blank())
  }
  return(p)
}

F.plotMultipleLines <- function(filelines,xtext=FALSE){
  p <- list()
  for (i in 1:nrow(filelines)){
    p[[i]] <- F.plotOneline(filelines[i,],xtext = xtext)
  }
  #p[[i+1]] <- F.plotOneline(filelines[i+1,], xtext = TRUE)
  return(grid.arrange(grobs=p,ncol=1))
}

#"intron","CDS","mito","wholeGap7","zChr"
filelines = df_files[df_files$baseType == 'mito'& df_files$seed == 1000,]
F.plotMultipleLines(filelines = filelines,xtext = TRUE)

df_files2 = df_files
df_files = df_files[df_files$population <= 6,]
pdf('/home/xcao/w/2018Calephelis/20181231Structure/26/26rawsoni_seed3000.pdf',width = 8, height = 16)
filelines = df_files[df_files$baseType == 'mito' & df_files$seed == 3000,]
F.plotMultipleLines(filelines = filelines, xtext=TRUE)



filelines = df_files[df_files$baseType == 'zChr'& df_files$seed == 3000,]
F.plotMultipleLines(filelines = filelines, xtext=TRUE)

filelines = df_files[df_files$baseType == 'CDS'& df_files$seed == 3000,]
F.plotMultipleLines(filelines = filelines, xtext=TRUE)


filelines = df_files[df_files$baseType == 'wholeGap7'& df_files$seed == 3000,]
F.plotMultipleLines(filelines = filelines, xtext=TRUE)

filelines = df_files[df_files$baseType == 'intron'& df_files$seed == 3000,]
F.plotMultipleLines(filelines = filelines, xtext=TRUE)

filelines = df_files[df_files$baseType == 'zChrCDS'& df_files$seed == 3000,]
F.plotMultipleLines(filelines = filelines, xtext=TRUE)

dev.off()
```

## treemix

### treemix 35 C. rawsoni group

#### filter sites with non-gap ratio 0.8, basekeepMinCount 4 minBaseType 1

```
txt = 'python /home/xcao/p/xiaolongTools/utils/fastaAlignmentFilter.py -i {file_in} -o {file_out} -t 32 -b 4 -g 0.8 -B 1 &'
workfolder = '/home/xcao/w/2018Calephelis/20190101rawsoni/Treemix/35groups'
files = '''/home/xcao/w/2018Calephelis/20190101rawsoni/Calephelis35.CDS
/home/xcao/w/2018Calephelis/20190101rawsoni/Calephelis35.intron
/home/xcao/w/2018Calephelis/20190101rawsoni/Calephelis35.wholeGap7
/home/xcao/w/2018Calephelis/20190101rawsoni/Calephelis35.zChr
'''.split()
import os
for f in files:
    basename = os.path.basename(f)
    file_out = os.path.join(workfolder,basename+'.filter')
    print(txt.format(file_in=f, file_out=file_out))
    
```

#### convert to Treemix input

```
txt = '''python3 /home/xcao/p/xiaolongTools/utils/fasta2TreemixInput_withGroups.py -i {file_in} -g {file_group} -o {file_out} -N 1 -G 0.8 &'''

workfolder = '/home/xcao/w/2018Calephelis/20190101rawsoni/Treemix/35groups'
files = '''/home/xcao/w/2018Calephelis/20190101rawsoni/Treemix/35groups/Calephelis35.CDS.filter
/home/xcao/w/2018Calephelis/20190101rawsoni/Treemix/35groups/Calephelis35.intron.filter
/home/xcao/w/2018Calephelis/20190101rawsoni/Treemix/35groups/Calephelis35.wholeGap7.filter
/home/xcao/w/2018Calephelis/20190101rawsoni/Treemix/35groups/Calephelis35.zChr.filter
'''.split()
file_group = '/home/xcao/w/2018Calephelis/20190101rawsoni/Treemix/35groups/sample2group'

import os
for f in files:
    basename = os.path.basename(f)
    file_out = os.path.join(workfolder,basename+'.treemix')
    os.system(txt.format(file_in=f, file_group=file_group, file_out=file_out))

```

#### run treemix
```

txt = '''/home/xcao/p/treemix/treemix -i {file_in} -o {file_out} -root outgroup &
'''
files = '''/home/xcao/w/2018Calephelis/20190101rawsoni/Treemix/35groups/Calephelis35.CDS.filter.treemix.gz
/home/xcao/w/2018Calephelis/20190101rawsoni/Treemix/35groups/Calephelis35.intron.filter.treemix.gz
/home/xcao/w/2018Calephelis/20190101rawsoni/Treemix/35groups/Calephelis35.wholeGap7.filter.treemix.gz
/home/xcao/w/2018Calephelis/20190101rawsoni/Treemix/35groups/Calephelis35.zChr.filter.treemix.gz
'''.split()

import os
for f in files:
    file_out = '/home/xcao/w/2018Calephelis/20190101rawsoni/Treemix/35groups/result/' + os.path.basename(f)[:-3]
    print(txt.format(file_in=f,file_out=file_out))
```

#### plot

```
source('/home/xcao/p/treemix/treemix-1.13/src/plotting_funcs.R')
library(glue)

pdf('/home/xcao/w/2018Calephelis/20190101rawsoni/Treemix/35groups/Calephelis35.pdf',width = 8,height = 11)

for (key in c('CDS','intron','wholeGap7','zChr')) {
  #print(key)
  filename.pre <- c('/home/xcao/w/2018Calephelis/20190101rawsoni/Treemix/35groups/result/Calephelis35.{key}.filter.treemix')

    filename <- paste0(glue(filename.pre))
    # print(filename)
    plot_tree(filename,maintitle = paste0(key,' '))
}

dev.off()
```

## careful study of Rawsoni

### Barcode and error detection

#### process and split barcodes from Nick
```
filename = '/home/xcao/w/2018Calephelis/summary/Calephelis_20181230_checked-barcodes.txt'

ls = open(filename).readlines()
ls = [e for e in ls if len(e)>1]

workfolder = '/home/xcao/w/2018Calephelis/20190110BarcodeChecking/'
import os
os.chdir(workfolder)
if not os.path.exists('barcodes'):
    os.mkdir('barcodes')
fout = open(workfolder+'id2notes','w')
for e in ls:
    annotation, seq = e.split()
    if '|' not in annotation:
        continue
    sample_id = annotation.split('|')[1]
    if sample_id.startswith('NVG-'):
        sample_id = sample_id[4:]
    fout.write(sample_id+'\t'+annotation+'\n')
    open(workfolder+'barcodes/'+sample_id,'w').write('>'+sample_id+'\n'+seq+'\n')
```

#### Extract and combine fastq to fasta

```
import pandas as pd
import os
import glob

folderBioHPC = '/work/archive/biophysics/Nick_lab/s185491/Celastrina/fq'
folder_fq = '/archive/butterfly/ready_fastq/'
folder_summary = '/home/xcao/w/2018Calephelis/summary/'

file_summary = os.path.join(folder_summary,'20181217Calephelis.xlsx')
df_summary = pd.read_excel(file_summary,sheet_name='Sheet1')
df_summary = df_summary[~df_summary.PC1_35_CDS.isna()]
sample_prefix = [e[1:] for e in df_summary['ID']]
print('unique sample prefix', len(set(sample_prefix)))

files_fq = glob.glob(folder_fq+'*')
files_use = [e for e in files_fq if os.path.basename(e).split('_')[0] in sample_prefix]

dc_fileuse = {}
for e in files_use:
    sample_id = os.path.basename(e).split('_')[0]
    if sample_id not in dc_fileuse:
        dc_fileuse[sample_id] = []
    dc_fileuse[sample_id].append(e)

from Bio import SeqIO
outfolder = '/home/xcao/w/2018Calephelis/20190110BarcodeChecking/fa/'
def fq2fa(k,files):
    fout = open(outfolder+k,'w')
    for f in files:
        for seq in SeqIO.parse(f,'fastq'):
            fout.write('>'+seq.id+'\n'+str(seq.seq)+'\n')
    fout.close()
from multiprocessing import Pool
pool = Pool(32)
pool.starmap(fq2fa, dc_fileuse.items())
pool.close()

```

#### transfer to bioHPC
```
scp -r /home/xcao/w/2018Calephelis/20190110BarcodeChecking/ s185491@nucleus.biohpc.swmed.edu:/work/biophysics/s185491/20181226Calephelis/
 scp -r /home/xcao/w/2018Calephelis/20190110BarcodeChecking/fa/ s185491@nucleus.biohpc.swmed.edu:/work/biophysics/s185491/20181226Calephelis/20190110BarcodeChecking/
```

#### blast
for each barcode, make a database with single seq. Blast with fasta input, first split to 32 parts, and combine the output

Stop testing this method. too slow.

#### jellyfish based barcode checking
```




# check existence of barcode files
import os
folder_barcode = '/work/biophysics/s185491/20181226Calephelis/20190110BarcodeChecking/barcodes/'
folder_fa = '/work/biophysics/s185491/20181226Calephelis/20190110BarcodeChecking/fa/'
fa_ids = os.listdir(folder_fa)
barcode_ids = os.listdir(folder_barcode)
sample_ids = open('/work/biophysics/s185491/20181226Calephelis/20190110BarcodeChecking/usingIDs').read().split()
print(len([e for e in sample_ids if e in barcode_ids])) #barcode for all 35 using file exists


#generate qsub scripts
folder_result = '/work/biophysics/s185491/20181226Calephelis/20190110BarcodeChecking/result/'
folder_fa = '/work/biophysics/s185491/20181226Calephelis/20190110BarcodeChecking/fa/'
folder_cmds = '/work/biophysics/s185491/20181226Calephelis/20190110BarcodeChecking/cmds/'
file_allbarcode = '/work/biophysics/s185491/20181226Calephelis/20190110BarcodeChecking/allBarcodes/intermediate_barcodes_20181231.fa'
    
qsub_txt = '''#!/bin/bash
#SBATCH --job-name=barcode
#SBATCH --partition=32GB
#SBATCH --nodes=1
#SBATCH --ntasks=32
#SBATCH --time=5-00:00:00
#SBATCH --output=0.%j.out
#SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL

python /home2/s185491/p/xiaolongTools/utils/seq/checkBarcode.py -b {file_allbarcode} -i {folder_fa}/{sample_id} -T {sample_id} -o {folder_result}/{sample_id}

'''

for sample_id in sample_ids:
    fout = open(folder_cmds+sample_id,'w')
    fout.write(qsub_txt.format(folder_result=folder_result,  file_allbarcode=file_allbarcode, folder_fa=folder_fa,sample_id=sample_id))
    fout.close()





```

### diagnostic characters of species

#### find genes

```
python /home/xcao/p/xiaolongTools/utils/speciesCharacter/characterFinderFromAlignment.py -a /home/xcao/w/2018Calephelis/20190101rawsoni/Calephelis35.CDS -l /home/xcao/w/genomes/Calephelis/3574_assembly_v1_withMito.alignment.exon.loc -A /home/xcao/w/genomes/Calephelis/cne_table_v1 -t /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/ExaML_result.Calephelis35.CDS.all5.sum.nwk.renamed -i /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/table_arizonensis -o /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/20190128Feature_arizonensis

python /home/xcao/p/xiaolongTools/utils/speciesCharacter/characterFinderFromAlignment.py -a /home/xcao/w/2018Calephelis/20190101rawsoni/Calephelis35.CDS -l /home/xcao/w/genomes/Calephelis/3574_assembly_v1_withMito.alignment.exon.loc -A /home/xcao/w/genomes/Calephelis/cne_table_v1 -t /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/ExaML_result.Calephelis35.CDS.all5.sum.nwk.renamed -i /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/table_freemani -o /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/20190128Feature_freemani -S 0.8 -C 0.8 -D 0.7

python /home/xcao/p/xiaolongTools/utils/speciesCharacter/characterFinderFromAlignment.py -a /home/xcao/w/2018Calephelis/20190101rawsoni/Calephelis35.CDS -l /home/xcao/w/genomes/Calephelis/3574_assembly_v1_withMito.alignment.exon.loc -A /home/xcao/w/genomes/Calephelis/cne_table_v1 -t /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/ExaML_result.Calephelis35.CDS.all5.sum.nwk.renamed -i /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/table_freemani_nuevoleon -o /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/20190128Feature_freemani_nuevoleon -S 0.8 -C 0.8 -D 0.8

python /home/xcao/p/xiaolongTools/utils/speciesCharacter/characterFinderFromAlignment.py -a /home/xcao/w/2018Calephelis/20190101rawsoni/Calephelis35.CDS -l /home/xcao/w/genomes/Calephelis/3574_assembly_v1_withMito.alignment.exon.loc -A /home/xcao/w/genomes/Calephelis/cne_table_v1 -t /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/ExaML_result.Calephelis35.CDS.all5.sum.nwk.renamed -i /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/table_nuevoleon -o /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/20190128Feature_nuevoleon

python /home/xcao/p/xiaolongTools/utils/speciesCharacter/characterFinderFromAlignment.py -a /home/xcao/w/2018Calephelis/20190101rawsoni/Calephelis35.CDS -l /home/xcao/w/genomes/Calephelis/3574_assembly_v1_withMito.alignment.exon.loc -A /home/xcao/w/genomes/Calephelis/cne_table_v1 -t /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/ExaML_result.Calephelis35.CDS.all5.sum.nwk.renamed -i /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/table_rawsoni -o /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/20190128Feature_rawsoni

python /home/xcao/p/xiaolongTools/utils/speciesCharacter/characterFinderFromAlignment.py -a /home/xcao/w/2018Calephelis/20190101rawsoni/Calephelis35.CDS -l /home/xcao/w/genomes/Calephelis/3574_assembly_v1_withMito.alignment.exon.loc -A /home/xcao/w/genomes/Calephelis/cne_table_v1 -t /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/ExaML_result.Calephelis35.CDS.all5.sum.nwk.renamed -i /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/table_freemani_rawsoni -o /home/xcao/w/2018Calephelis/20190101rawsoni/20190128Feature/20190128Feature_freemani_rawsoni -S 0.8 -C 0.8 -D 0.8 -M 0.8
```

### clean up 15109F05
maybe contaminated by 15114F01_Arumecla_aruma

#### Jellyfish 25mer count, inchworm Arumecla_aruma

This method does not work. Inchworm cannot assemble the kmers well

keep contig longer than 100bp

```
jellyfish count -m 25 -s 2G -t 48 -C -o /home/xcao/w/2018Calephelis/20190207cleanup15109F05/Arumecla_aruma/Arumecla_aruma.25mer.jf /archive/butterfly/ready_fastq/15108F12* /archive/butterfly/ready_fastq/15114E11* /archive/butterfly/ready_fastq/15114E12* /archive/butterfly/ready_fastq/15114F01* /archive/butterfly/ready_fastq/15114F02* /archive/butterfly/ready_fastq/15114F03* /archive/butterfly/ready_fastq/15114F04*
jellyfish dump /home/xcao/w/2018Calephelis/20190207cleanup15109F05/Arumecla_aruma/Arumecla_aruma.25mer.jf >/home/xcao/w/2018Calephelis/20190207cleanup15109F05/Arumecla_aruma/Arumecla_aruma.25mer.fa


```

#### kmer counting and sort by kmers
use 21mer


## study several groups
20190208  
* C.nemesis-group
* C.nilus-group
* C.velutina-group
* C.burgeri
* C.fulmen-group
* C.rawsoni-group

### check existance of files

#### fq, map, stats
```
import pandas as pd
import os
import glob
import itertools
import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/mapFile')
import mapfileIO

folder_fq = '/archive/butterfly/ready_fastq/'



file_summary = '/work0/xcao/2018Calephelis/summary/20190207Calephelis.xlsx'
df_summary = pd.read_excel(file_summary,sheet_name='Sheet1',dtype=str)
sample_prefix = [e[1:] for e in df_summary['ID']]
print('unique sample prefix', len(set(sample_prefix)))
dc_sample2fq = {e:[] for e in sample_prefix}
for k in dc_sample2fq:
    fq = glob.glob('/archive/butterfly/ready_fastq/'+k+'_*')
    dc_sample2fq[k] = fq
dc_sample2map = {e:[] for e in sample_prefix}
for k in dc_sample2map:
    fmap = glob.glob('/archive/butterfly/maps/debiased/'+k+'_3574_assembly_v1_withMito_snp_step2.map')
    dc_sample2map[k] = fmap
dc_sample2stats = {e:[] for e in sample_prefix}
for k in dc_sample2stats:
    fstats = glob.glob('/archive/butterfly/unbias_pipeline_info/step4_postprocessing/final_stats/'+k+'_3574_assembly_v1_withMito_stat.report')
    dc_sample2stats[k] = fstats
sample_fq = [k for k,v in dc_sample2fq.items() if len(v)>0]
sample_map = [k for k,v in dc_sample2map.items() if len(v)>0]
sample_stats = [k for k,v in dc_sample2stats.items() if len(v)>0]
print(len(sample_fq),'samples with fastq')
print(len(sample_map), 'samples with map')
print(len(sample_stats), 'samples with stats')
samples_nomap = [e for e in sample_fq if e not in sample_map]
print('\n'.join(samples_nomap))

#convert map files to npInt8
dc_sample2np = {k:[] for k in dc_sample2map}
for k in dc_sample2np:
    file = glob.glob('/home/xcao/w/2018Calephelis/summary/map.npInt8.0/'+k+'_3574_assembly_v1_withMito_snp_step2.map')
    dc_sample2np[k] = file

file_map4convert = [v for k,v in dc_sample2map.items() if len(dc_sample2np[k]) == 0 and len(v)>0]
file_map4convert = list(itertools.chain.from_iterable(file_map4convert))
mapfileIO.read2Int8s(file_map4convert,strand=0, changeGap=True, threads=16, outfolder='/home/xcao/w/2018Calephelis/summary/map.npInt8.0/')
mapfileIO.read2Int8s(file_map4convert,strand=2, changeGap=True, threads=16, outfolder='/home/xcao/w/2018Calephelis/summary/map.npInt8.2/')

#for sample 3574, convert genome to mapfile
file_genome = '/home/xcao/w/genomes/Calephelis/3574_assembly_v1_withMito.fa'
from Bio import SeqIO
l = []
for s in SeqIO.parse(file_genome,'fasta'):
    if s.id != '3574_mito':
        l.append(str(s.seq))
l = ''.join(l)
fout = open('/home/xcao/w/2018Calephelis/summary/3574_3574_assembly_v1_withMito_snp_step2.map','w')
fout.write('\n'.join(list(l)))
fout.write('\n')
fout.close()
mapfileIO.read2Int8s(['/home/xcao/w/2018Calephelis/summary/3574_3574_assembly_v1_withMito_snp_step2.map'],strand=0, changeGap=True, threads=16, outfolder='/home/xcao/w/2018Calephelis/summary/map.npInt8.0/')
sample_map.append('3574')


file_summary = '/work/xcao/summary/All-sequenced.xlsx'
sheets = '''C.nemesis-group
C.nilus-group
C.velutina-group
C.burgeri
C.fulmen-group
C.rawsoni-group
'''.split()

dcdf = dict.fromkeys(sheets)
for sheet_name in sheets:
    df_summary = pd.read_excel(file_summary,sheet_name=sheet_name,dtype=str)
    df_summary = df_summary[df_summary['DNA number'] != 'nan']
    dcdf[sheet_name] = df_summary
for sheet_name, df_summary in dcdf.items():
    sample_prefix = [e.replace('NVG-','') for e in df_summary['DNA number']]
    print(sheet_name, 'unique sample prefix', len(set(sample_prefix)))
    sample_withmap = [e for e in sample_prefix if e in sample_map]
    sample_withhoutmap = [e for e in sample_prefix if e not in sample_map]
    print('samples with map file', len(sample_withmap))
    print('samples without map file', sample_withhoutmap)
    print('samples without fq', [e for e in sample_withhoutmap if e not in sample_fq])
    fout = open('/home/xcao/w/2018Calephelis/summary/samples_'+sheet_name,'w')
    fout.write('\n'.join(sample_prefix))
    fout.close()
    
```

#### result for 6 groups
```
C.nemesis-group unique sample prefix 32
samples with map file 36
samples without map file ['3751']
samples without fq ['3751']
C.nilus-group unique sample prefix 34
samples with map file 34
samples without map file []
samples without fq []
C.velutina-group unique sample prefix 16
samples with map file 13
samples without map file ['18043F05', '18098H01', '18098H02']
samples without fq []
C.burgeri unique sample prefix 9
samples with map file 9
samples without map file []
samples without fq []
C.fulmen-group unique sample prefix 35
samples with map file 33
samples without map file ['18098G10', '18098G12']
samples without fq []
C.rawsoni-group unique sample prefix 36
samples with map file 35
samples without map file ['18048A01']
samples without fq []

```

### get CDS sequences and zChrCDS

#### all samples

```
import sys
sys.path.append('/home2/s185491/p/xiaolongTools/utils/mapFile')
import mapfileIO
import glob

folder = '/project/biophysics/Nick_lab/s185491/Calephelis/summary/map.npInt8.0/'
files = glob.glob(folder+'*')
print('total map npInt8 files', len(files))

file_filter = '/project/biophysics/Nick_lab/s185491/Calephelis/summary/map.sitesInfo/gff.0inter.2intron.6CDS'
outfile = '/project/biophysics/Nick_lab/s185491/Calephelis/phylo/allseq/Calephelis270.CDS'
mapfileIO.npInt8stoFastas(files, threads=48, output=outfile, headerFun=lambda x:x.split('_')[0], npFilter=file_filter, npFilterFun=lambda x:x==6, merged=True)

file_filter = '/project/biophysics/Nick_lab/s185491/Calephelis/summary/map.sitesInfo/zChrCDSFilter'
outfile = '/project/biophysics/Nick_lab/s185491/Calephelis/phylo/allseq/Calephelis270.zChrCDS'
mapfileIO.npInt8stoFastas(files, threads=32, output=outfile, headerFun=lambda x:x.split('_')[0], npFilter=file_filter, npFilterFun=None, merged=True)
```

#### get sequences for each group and split CDS 100 parts, zChrCDS 5 parts
```
file_CDS = '/project/biophysics/Nick_lab/s185491/Calephelis/phylo/allseq/Calephelis270.CDS'
file_zChrCDS = '/project/biophysics/Nick_lab/s185491/Calephelis/phylo/allseq/Calephelis270.zChrCDS'
groups = '''C.nemesis-group
C.nilus-group
C.velutina-group
C.burgeri
C.fulmen-group
C.rawsoni-group
'''.split()

import os
from Bio import SeqIO
seqs_CDS = list(SeqIO.parse(file_CDS,'fasta'))
seqs_zChrCDS = list(SeqIO.parse(file_zChrCDS,'fasta'))
for g in groups:
    outfolder = '/project/biophysics/Nick_lab/s185491/Calephelis/phylo/'+g
    if not os.path.exists(outfolder):
        os.makedirs(outfolder)
    samples = open('/project/biophysics/Nick_lab/s185491/Calephelis/summary/samples_'+g).read().split()
    with open(os.path.join(outfolder,'CDS.fa'),'w') as f:
        for s in seqs_CDS:
            if s.id in samples:
                f.write('>' + s.id +'\n'+str(s.seq)+'\n')
    with open(os.path.join(outfolder,'zChrCDS.fa'),'w') as f:
        for s in seqs_zChrCDS:
            if s.id in samples:
                f.write('>' + s.id +'\n'+str(s.seq)+'\n')
for g in groups:
    outfolder = '/project/biophysics/Nick_lab/s185491/Calephelis/phylo/'+g
    cmd = 'cd {outfolder} && python /home2/s185491/p/xiaolongTools/utils/splitFastaAlignments2NPartsContinuingEqualStepOrRandom.py -i CDS.fa -m 1 -t 48 -b 10 -N 100'.format(outfolder=outfolder)
    os.system(cmd)
    cmd = 'cd {outfolder} && python /home2/s185491/p/xiaolongTools/utils/splitFastaAlignments2NPartsContinuingEqualStepOrRandom.py -i zChrCDS.fa -m 1 -t 48 -b 10 -N 5'.format(outfolder=outfolder)
    os.system(cmd)

```

#### run ExaML

```
txt_qsub = '''#!/bin/bash
#SBATCH --job-name={n}
#SBATCH --partition=32GB
#SBATCH --nodes=1
#SBATCH --ntasks=32
#SBATCH --time=5-00:00:00
#SBATCH --output=0.%j.out
####SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL

module load openmpi/intel/3.1.1

python3 /home2/s185491/p/xiaolongTools/utils/runExaML_withFastaForLargeFiles.py -i {filename} -T 32
'''

import glob
files = glob.glob('/project/biophysics/Nick_lab/s185491/Calephelis/phylo/*/*.fa.split*')

import os
for filename in files:
    basename = os.path.basename(filename)
    groupname = os.path.basename(os.path.dirname(filename))
    fout = open('/project/biophysics/Nick_lab/s185491/Calephelis/phylo/qsub/ExaML'+groupname+basename,'w')
    fout.write(txt_qsub.format(filename=filename,n=basename))
    fout.close()

groups = '''C.nemesis-group
C.nilus-group
C.velutina-group
C.burgeri
C.fulmen-group
C.rawsoni-group
'''.split()

import os
for g in groups:
    outfolder = '/project/biophysics/Nick_lab/s185491/Calephelis/phylo/'+g
    cmd = '''cd {outfolder} && \
    cat ExaML_result.CDS.fa.split100.* >ExaML_result.CDS.all &&\
    /home2/s185491/p/anaconda3/anaconda520/bin/sumtrees.py -f 0.00 ExaML_result.CDS.all>ExaML_result.CDS.sum &&\
    python /home2/s185491/p/xiaolongTools/utils/trees/sumtreesNEXUS2newick.py -i ExaML_result.CDS.sum &&\
    /home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py  ExaML_result.CDS.sum.nwk &&\
    java -Xmx3000m -jar /home2/s185491/p/Astral/astral.5.6.2.jar -i ExaML_result.CDS.all -o ExaML_result.CDS.astral &&\
    /home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py  ExaML_result.CDS.astral '''.format(outfolder=outfolder)
    os.system(cmd)

for g in groups:
    outfolder = '/project/biophysics/Nick_lab/s185491/Calephelis/phylo/'+g
    cmd = '''cd {outfolder} && \
    cat ExaML_result.zChrCDS.fa.split5.* >ExaML_result.zChrCDS.all &&\
    /home2/s185491/p/anaconda3/anaconda520/bin/sumtrees.py -f 0.00 ExaML_result.zChrCDS.all>ExaML_result.zChrCDS.sum &&\
    python /home2/s185491/p/xiaolongTools/utils/trees/sumtreesNEXUS2newick.py -i ExaML_result.zChrCDS.sum &&\
    /home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py  ExaML_result.zChrCDS.sum.nwk &&\
    java -Xmx3000m -jar /home2/s185491/p/Astral/astral.5.6.2.jar -i ExaML_result.zChrCDS.all -o ExaML_result.zChrCDS.astral &&\
    /home2/mtang/anaconda2/bin/python /work/biophysics/mtang/SNP_calling/scripts/rename_tree_fullname.py  ExaML_result.zChrCDS.astral '''.format(outfolder=outfolder)
    os.system(cmd)

```